In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/bdb/2024')
%cd /content/drive/My Drive/bdb/2024

project_dir = '/content/drive/My Drive/bdb/2024/data/'


Mounted at /content/drive
/content


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import log_loss
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
from datetime import datetime
import smtplib
import sys
import regex

In [ ]:
CARRIERS = {
    "att": "@mms.att.net",
    "tmobile": "@tmomail.net",
    "verizon": "@vtext.com",
    "sprint": "@messaging.sprintpcs.com"
}

EMAIL = 'jsiekmann@gmail.com'
PASSWORD = 'xscnzybxylzixmjg'

def send_message(phone_number, carrier, message):
    recipient = phone_number + CARRIERS[carrier]
    auth = (EMAIL, PASSWORD)

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(auth[0], auth[1])

    server.sendmail(auth[0], recipient, message)

In [ ]:
send_message('7202725275','att','biscuits are done!')

In [3]:
tackles = pd.read_csv(f'{project_dir}tackles.csv')
plays = pd.read_csv(f'{project_dir}plays.csv')
players = pd.read_csv(f"{project_dir}players.csv")
run_plays = pd.read_csv(f'{project_dir}run_plays.csv')
run_plays_adj = run_plays.values.tolist()

end_event = ['tackle','out_of_bounds']

In [ ]:
end_event = ['tackle','out_of_bounds']

run_plays = [list(z) \
             for z in zip(plays[~plays['playDescription'].str.contains('pass')]['gameId'].to_list(),\
                          plays[~plays['playDescription'].str.contains('pass')]['playId'].to_list())]
run_plays_remove = [[2022091801,63],\
                    [2022101605,2970]\
                    [2022102307, 1505]\
                   ]
[run_plays.remove(x) for x in run_plays_remove]

In [ ]:
df_ = pd.read_csv(f'{project_dir}games_processed_3.csv')

In [ ]:
df = df_.copy()

In [ ]:
df_clean = df[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time',
       'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a', 'dis', 'o',
       'dir', 'event', 'position', 'tackle', 'assist']]

In [ ]:
df.columns

In [ ]:
week_gameIds = week.gameId.unique().tolist()

In [ ]:
run_plays_total = []
for gid in tqdm(week_gameIds):
    print('Game ID', gid)
    game = week.loc[week['gameId']==gid].copy()

    #print('filtering run_plays',end='\r', flush=True)
    game_ = pd.DataFrame()
    for i in tqdm(range(len(run_plays)),leave=False):
        game_ = pd.concat([game_,game[(game['gameId'] == run_plays[i][0]) & (game['playId'] == run_plays[i][1])]])

    run_plays_adj = game_[game_['event'].isin(end_event)]['playId'].unique().tolist()

    # Discard plays under 3 seconds
    under_time = []
    for pid in run_plays_adj:
        if game_[game_['playId']==pid]['frameId'].max() < 30:
            under_time.append(pid)
    [run_plays_adj.remove(x) for x in under_time]

    run_plays_total.extend([list(a) for a in zip([gid]*len(run_plays_adj),run_plays_adj)])

pd.DataFrame(run_plays_total,columns=['gameId','playId']).to_csv(f'{project_dir}run_plays.csv',index=False)

In [ ]:
week = pd.read_csv(f"{project_dir}/tracking_week_{1}.csv")

In [ ]:
week.head()

In [ ]:
week = pd.DataFrame()
for w in range(1,10):
    print('processing week', w)
    if week.empty:
        week = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
        week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
        week = week.merge(tackles,  how='left', left_on=['gameId','playId','nflId'], right_on = ['gameId','playId','nflId'])
        week = week.merge(plays,how='left',on=['gameId','playId'])
    else:
        temp = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
        temp = temp.merge(players.loc[:, ['nflId', 'position']], how='left')
        temp = temp.merge(tackles,  how='left', left_on=['gameId','playId','nflId'], right_on = ['gameId','playId','nflId'])
        temp = temp.merge(plays,how='left',on=['gameId','playId'])
        week = pd.concat([week,temp])

In [ ]:
df_clean = pd.DataFrame()
week.sort_values(['gameId','playId','frameId','nflId']).reset_index(drop=True)
new_game = 0
game_csv_list = []
for gid in tqdm(run_plays_adj):
    if new_game % 50 == 0:
        df_clean.to_csv(f'{project_dir}{gid[0]}_{gid[1]}.csv')
        game_csv_list.append([gid[0],gid[1]])
        df_clean = pd.DataFrame()
    temp = week[(week['gameId']==gid[0])&(week['playId']==gid[1])].copy()
    df_clean = pd.concat([df_clean,calibrate_coords(temp)])
    new_game += 1

df_clean = pd.concat([df_clean,calibrate_coords(temp)])
df_clean.to_csv(f'{project_dir}week_final.csv')
pd.DataFrame(game_csv_list,columns=['games','plays']).to_csv(f'{project_dir}game_csv_list.csv',index=False)

In [ ]:
df_clean = pd.DataFrame()
for gid in tqdm(game_csv_list):
    df_clean = pd.concat([df_clean,pd.read_csv(f'{project_dir}{gid[0]}_{gid[1]}.csv')])

df_clean = pd.concat([df_clean,pd.read_csv(f'{project_dir}week_final.csv')])

In [ ]:
df_clean.to_csv(f'{project_dir}df_clean.csv',index=False)

In [ ]:
df_clean[['gameId','playId','nflId','frameId']] = df_clean[['gameId','playId','nflId','frameId']].astype('Int64')
df_clean

In [ ]:
df_clean = df_clean.iloc[: , 1:]
df_clean.head()

In [ ]:
df_clean = df_clean[df_clean['displayName']!='football']
df_clean['tackle'] = df_clean['tackle'].fillna(0)
df_clean['assist'] = df_clean['assist'].fillna(0)
df_clean['both'] = df_clean['tackle'] + df_clean['assist']
df_clean = df_clean.sort_values(['gameId','playId','frameId','nflId']).reset_index(drop=True)

In [ ]:
df_clean.shape

In [ ]:
df_clean.to_csv(f'{project_dir}df_clean.csv',index=False)

In [4]:
df_clean = pd.read_csv(f'{project_dir}df_clean.csv')

<ipython-input-4-3ac199da33d2>:1: DtypeWarning: Columns (35,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clean = pd.read_csv(f'{project_dir}df_clean.csv')


In [5]:
bc_new = df_clean.loc[df_clean['nflId']==df_clean['ballCarrierId'], ['nflId','gameId', 'playId', 'frameId', 'x','y', 's','a','dir']].rename(columns={'x':'x_bc','y':'y_bc','s':'s_bc', 'a':'a_bc','dir':'dir_bc'}).copy(deep=True)
bc_new = bc_new.sort_values(['gameId','playId','frameId']).reset_index(drop=True)

In [ ]:
bc_new.head()

In [ ]:
bc_new.shape

In [7]:
bc = pd.DataFrame()
for gid in tqdm(run_plays_adj):
    #df_play['bc_xdist'] = 0
    df_play = bc_new.loc[(bc_new['gameId']==gid[0])&(bc_new['playId']==gid[1])].copy()
    df_play.reset_index(drop=True,inplace=True)
    df_play['s_x_bc'] = np.sin(np.radians(df_play['dir_bc'])) * df_play['s_bc']
    df_play['ds_x_bc'] = df_play.loc[:,'s_x_bc'].sub(df_play.loc[:,'s_x_bc'].shift()).fillna(0)
    df_play['ds_bc'] = df_play.loc[:,'s_bc'].sub(df_play.loc[:,'s_bc'].shift()).fillna(0)
    df_play['a_x_bc'] = np.sin(np.radians(df_play['dir_bc'])) * df_play['a_bc']
    df_play['da_x_bc'] = df_play.loc[:,'a_x_bc'].sub(df_play.loc[:,'a_x_bc'].shift()).fillna(0)
    df_play['da_bc'] = df_play.loc[:,'a_bc'].sub(df_play.loc[:,'a_bc'].shift()).fillna(0)
    df_play['x_bc_diff'] = df_play.loc[:,'x_bc'].sub(df_play.loc[:,'x_bc'].shift()).fillna(0)
    df_play['bc_xdist'] = df_play['x_bc_diff'].cumsum()
    bc = pd.concat([bc,df_play])

bc.reset_index(drop=True,inplace=True)

In [ ]:
bc.describe()

In [ ]:
bc.head(55)

In [ ]:
bc.describe()

In [ ]:
bc['y_bound'] = bc['y_bc']
bc.loc[bc['y_bc'] > (160/3),'y_bound'] = (160/3)
bc.loc[bc['y_bc'] < 0,'y_bound'] = 0

bc['y_adj'] = np.abs(bc['y_bound'] - (160/6))
bc['d_y'] = 0
bc.loc[(bc['y_bound']<(160/6))&(bc['dir_bc'].between(90,270)),'d_y'] = 1
bc.loc[(bc['y_bound']>(160/6))&((bc['dir_bc'].between(0,90)|(bc['dir_bc'].between(270,360)))),'d_y'] = 1
ep1 = 1
ep2 = 0
bc['boundary'] = np.exp(bc['y_adj']-ep1*(ep2+160/6)) * bc['d_y']

In [ ]:
bc.shape

In [ ]:
bc.describe()

In [ ]:
bc.head()

In [ ]:
df = df_clean.copy()

In [6]:
df = df_clean.merge(bc,how='left',on=['gameId','playId','frameId'])
df = df.rename(columns={'nflId_x':'nflId'})

NameError: name 'bc' is not defined

In [ ]:
df.shape

In [ ]:
def players_between(x_a,y_a,x_b,y_b,x_bc,y_bc,r):

    x_a -= x_b
    y_a -= y_b
    x_bc -= x_b
    y_bc -= y_b
    a = (x_bc - x_a)**2 + (y_bc - y_a)**2
    b = 2*(x_a*(x_bc - x_a) + y_a*(y_bc - y_a))
    c = x_a**2 + y_a**2 - r**2
    quadr = b**2 - 4*a*c
    if quadr < 0:
        return False
    p1 = (-b + np.sqrt(quadr))/(2*a)
    p2 = (-b - np.sqrt(quadr))/(2*a)
    if (p1 > 0) & (p1 < 1):
        return True
    elif (p2 > 0) & (p2 < 1):
        return True
    else:
        return False


In [ ]:
player_radius = 1/3
df['between_bc'] = None
df['between_bc'] = df['between_bc'].astype('str')
for gid in tqdm(run_plays_adj):
    temp = df.loc[(df['gameId']==gid[0])&(df['playId']==gid[1])].copy()
    for fid in tqdm(temp['frameId'].unique(),leave=False):
        for pid1 in temp['nflId'].unique():
            between_list = []
            player = temp.loc[(temp['frameId']==fid)&(temp['nflId']==pid1)]
            if player['club'].values[0]==player['defensiveTeam'].values[0]:
                for pid2 in temp['nflId'].unique():
                    if pid1 != pid2:
                        player_other = temp.loc[(temp['frameId']==fid)&(temp['nflId']==pid2)]
                        #print(fid, player['displayName'].values[0],player_other['displayName'].values[0])
                        x_a = player['x'].values[0]
                        y_a = player['y'].values[0]
                        x_b = player_other['x'].values[0]
                        y_b = player_other['y'].values[0]
                        x_bc = player['x_bc'].values[0]
                        y_bc = player['y_bc'].values[0]
                        if players_between(x_a,y_a,x_b,y_b,x_bc,y_bc,player_radius):
                            between_list.append(pid2)
                            #df.loc[(df['gameId']==gid[0])&(df['playId']==gid[1])&(temp['frameId']==fid)&(temp['nflId']==pid1),'between_bc'] += 1
            print('between',between_list)
            if between_list:
                between = '_'
                for item in between_list:
                    between = between + f'{item}_'
                df.loc[(df['gameId']==gid[0])&(df['playId']==gid[1])&(temp['frameId']==fid)&(temp['nflId']==pid1),'between_bc'] = between
            #print(df.loc[(df['gameId']==gid[0])&(df['playId']==gid[1])&(temp['frameId']==fid)&(temp['nflId']==pid1),'between_bc'].values)


    df.loc[(df['gameId']==gid[0])&(df['playId']==gid[1])].to_csv(f'{project_dir}{gid[0]}_{gid[1]}_between.csv',index=False)
send_message('7202725275','att','biscuits are done!')

In [ ]:
df = df_clean[df_clean['club']==df_clean['defensiveTeam']].copy()

In [ ]:
df = df.sort_values(by=['nflId','gameId','playId','frameId'])

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{project_dir}df_12_31.csv',index=False)

In [ ]:
df = pd.read_csv(f'{project_dir}df_12_31.csv')

In [ ]:
df['dist_to_bc'] = np.sqrt(((df['x_bc'] - df['x'])**2) + ((df['y_bc'] - df['y'])**2))
df['s_to_bc'] = -1*(df.loc[:,'dist_to_bc'].sub(df.loc[:,'dist_to_bc'].shift()).fillna(0))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 's_to_bc'] = 0
#df.loc[df['frameId']==df['frame_min'],'s_to_bc'] = 0
df['ds_to_bc'] = -1*(df.loc[:,'s_to_bc'].sub(df.loc[:,'s_to_bc'].shift()).div(0.1).fillna(0))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 'ds_to_bc'] = 0
#df.loc[df['frameId']==df['frame_min'],'ds_to_bc'] = 0
#df.loc[df['frameId']==df['frame_min'],'a_x_bc'] = 0

In [ ]:
df['activation'] = 0
df['boundary_final'] = 0
df.loc[df['dist_to_bc']<3,'activation'] = (-(1/3)*df.loc[df['dist_to_bc']<3,'dist_to_bc'])+1   # linearly increases from zero to 1 once defender within 3 yards of football
df['boundary_final'] = df['activation'] * df['boundary']

In [ ]:
df['bc_x_min'] = df.groupby(['gameId','playId'])['x_bc'].transform('min')
df['bc_x_max'] = df.groupby(['gameId','playId'])['x_bc'].transform('max')
df['bc_x_perc'] = (df['x_bc'].sub(df['bc_x_min']))/(df['bc_x_max'].sub(df['bc_x_min']))

df['frame_min'] = df.groupby(['gameId','playId'])['frameId'].transform('min')
df['frame_max'] = df.groupby(['gameId','playId'])['frameId'].transform('max')
df['frame_perc'] = (df['frameId'].sub(df['frame_min']))/(df['frame_max'].sub(df['frame_min']))

In [ ]:
df['s_x_bc'] = np.sin(np.radians(df['dir_bc'])) * df['s_bc']
df['a_x_bc'] = np.sin(np.radians(df['dir_bc'])) * df['a_bc']

In [ ]:
df['y_diff'] = df['y_bc'] - df['y']
df.loc[df['y_diff']<0.01,'y_diff'] == 0.01
df['x_diff'] = df['x_bc'] - df['x']
df['slope'] = df['x_diff'].div(df['y_diff'])
df['o_to_bc'] = np.arctan(df['slope'])*(180/np.pi)
df['o_diff'] = abs(((df['o'] - df['o_to_bc'])+180) % 360 - 180)
df['o_diff_perc'] = (180-df['o_diff'])/180

df['dir_diff'] = abs(((df['dir'] - df['o_to_bc'])+180) % 360 - 180)
df['dir_diff_perc'] = (180-df['dir_diff'])/180
df['s_dir'] = df['dir_diff_perc'].mul(df['s'])

In [ ]:
print(df.shape)
df.head()

In [ ]:
bc_radius = 1
df['inside_zone'] = df.groupby(['gameId','playId','frameId'])['dist_to_bc'].transform(lambda x: (x<=bc_radius).sum())

In [ ]:
df.loc[df['dist_to_bc']>=bc_radius,'tackle'] = 0
df.loc[df['dist_to_bc']>=bc_radius,'assist'] = 0
df.loc[df['dist_to_bc']>=bc_radius,'both'] = 0

In [ ]:
min_limit = 0.01

df.loc[df['dist_to_bc']<min_limit,'dist_to_bc'] = min_limit
#df.loc[df['bc_xdist']<min_limit,'bc_xdist'] = min_limit
df.loc[df['s_bc']<min_limit,'s_bc'] = min_limit
#df.loc[df['a_x_bc']<min_limit,'a_x_bc'] = min_limit
df.loc[df['s_to_bc']<min_limit,'s_to_bc'] = min_limit
df.loc[df['s']<min_limit,'s'] = min_limit

#df['big'] = -df['a_x_bc']*df['s_dir']*df['o_diff_perc']/(df['dist_to_bc']+np.sqrt(df['bc_xdist']))
#df['big2'] = -df['ds_x_bc']*df['s_dir']*df['o_diff_perc']/(df['dist_to_bc']+np.sqrt(df['bc_xdist']))
#df['big3'] = df['s_dir']*df['o_diff_perc']/(df['dist_to_bc']*df['s_bc'])
#df['big4'] = df['s_to_bc']/df['dist_to_bc']
#df['big4'] = (df['o_diff_perc']*df['dir_diff_perc'])/(df['s_to_bc']*df['dist_to_bc']*df['s_bc']*df['s'])
df['big5'] = df['s_to_bc']*df['s_bc']

In [ ]:
df['big5'] = df['s_to_bc']*df['s_bc']
df['d_big5'] = -1*(df.loc[:,'big5'].sub(df.loc[:,'big5'].shift()))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 'd_big5'] = 0
df['d_big5_x'] = df.loc[:,'d_big5'].div(df.loc[:,'dist_to_bc'])

In [ ]:
df.d_big5_x.sort_values()

In [ ]:
df.to_csv(f'{project_dir}df_01_02.csv',index=False)

In [ ]:
df = pd.read_csv(f'{project_dir}df_01_01.csv')

In [ ]:
features = ['s','a','s_dir','dist_to_bc','s_bc','s_x_bc','a_bc','a_x_bc','o_diff_perc','dir_diff_perc','boundary_final','inside_zone','ds_x_bc','da_x_bc','ds_bc','da_bc','s_to_bc','d_big5_x']
target = 'both'

In [ ]:
model_data = df.dropna(subset=features+[target]).reset_index(drop=True)

In [ ]:
model_data.shape

In [ ]:
train.shape

In [ ]:
wks = [8,9]
temp = []
games_wk = []
for w in wks:
    week_ = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
    temp.append(week_['gameId'].unique().tolist())

for nested in temp:
    for n in nested:
        games_wk.append(n)

In [ ]:
test = model_data[model_data['gameId'].isin(games_wk)].copy()
x_test = xgb.DMatrix(test.loc[:,features],test[target])

train = model_data[~model_data['gameId'].isin(games_wk)].copy()
x_train = xgb.DMatrix(train.loc[:,features],train[target])

In [ ]:
train[features].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

test = model_data[model_data['gameId'].isin(games_wk)].copy()
test[['s','a','s_dir','dist_to_bc','bc_s','bc_s_x','bc_a','bc_a_x']] = scaler.fit_transform(test[['s','a','s_dir','dist_to_bc','bc_s','bc_s_x','bc_a','bc_a_x']])

train = model_data[~model_data['gameId'].isin(games_wk)].copy()
train[['s','a','s_dir','dist_to_bc','bc_s','bc_s_x','bc_a','bc_a_x']] = scaler.fit_transform(train[['s','a','s_dir','dist_to_bc','bc_s','bc_s_x','bc_a','bc_a_x']])


In [ ]:
train['both'].sum()

In [ ]:
train['both'].sum()/len(train)

In [ ]:
%pip install scikit-optimize

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#from category_encoders.target_encoder import TargetEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from collections import Counter

In [ ]:
estimators = [
    #('encoder', TargetEncoder()),
    ('clf', XGBClassifier(random_state=24)) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)

In [ ]:
search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0),
    'clf__scale_pos_weight': Real(10,15,'log-uniform')
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=10, scoring='roc_auc', random_state=24)

In [ ]:
# TRAIN
wks = np.arange(1,5)
temp = []
games_train = []
for w in wks:
    week_ = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
    temp.append(week_['gameId'].unique().tolist())

for nested in temp:
    for n in nested:
        games_train.append(n)
len(games_train)

In [ ]:
# TEST
wks = [5]
temp = []
games_test = []
for w in wks:
    week_ = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
    temp.append(week_['gameId'].unique().tolist())

for nested in temp:
    for n in nested:
        games_test.append(n)
len(games_test)

In [ ]:
test = df[df['gameId'].isin(games_test)].copy()
test.reset_index(drop=True,inplace=True)
x_test = test.loc[:,features]
y_test = test[target]

train = df[df['gameId'].isin(games_train)].copy()
train.reset_index(drop=True,inplace=True)
x_train = train.loc[:,features]
y_train = train[target]

In [ ]:
y_train.sum()/len(y_train)

In [ ]:
opt.fit(x_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.score(x_test, y_test)

In [ ]:
opt.predict(x_test)

In [ ]:
opt.predict_proba(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, PrecisionRecallDisplay, precision_recall_curve

roc_auc_score(y_test, opt.predict_proba(x_test)[:, 1])   # this is same as opt.score

In [ ]:
xgboost_step = opt.best_estimator_.steps[0]
xgboost_model = xgboost_step[1]
plot_importance(xgboost_model,max_num_features=20)

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, opt.predict_proba(x_test)[:, 1])
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
preds = opt.predict_proba(x_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, preds)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [ ]:
opt.best_params_

In [ ]:
y_pred = opt.predict(x_test)
y_pred_prob = opt.predict_proba(x_test)[:, 1]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot()

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, recall_score, precision_score

accuracy = accuracy_score(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

# Print the evaluation metrics
print("XGBoost Accuracy:", accuracy)
print("Kappa Score:", kappa)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

In [ ]:
def f1g(score_, pi_):
    return (score_ - pi_)/(score_*(1-pi_))

In [ ]:
pi_ = 0.03
recall_g = f1g(recall, pi_)
precision_g = f1g(precision, pi_)
f1_gain = (recall_g + precision_g)/2

print('Recall Gain:', recall_g)
print('Precision Gain:', precision_g)
print('F1 Gain:', f1_gain)

In [ ]:
test_final = pd.DataFrame(y_pred_prob).rename(columns={0:'y_pred'}).join(test)
test_final['frameId'] = test_final['frameId'].astype('float32')

In [ ]:
df.to_csv(f'{project_dir}df_bigs.csv',index=False)

In [ ]:
df.columns

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(f'{project_dir}df_bigs.csv')

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
min_limit = 0.1
df.loc[df['dist_to_bc']<min_limit,'dist_to_bc'] = min_limit

In [ ]:
df['big6'] = df['s_to_bc']*df['s_x_bc']*df['s']
df['d_big6'] = -1*(df.loc[:,'big6'].sub(df.loc[:,'big6'].shift()))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 'd_big6'] = 0
df['d_big6_x'] = df.loc[:,'d_big6'].div(df.loc[:,'dist_to_bc'])

In [ ]:
df['big7'] = df['s_x_bc']*df['s']
df['d_big7'] = -1*(df.loc[:,'big7'].sub(df.loc[:,'big7'].shift()))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 'd_big7'] = 0
df['d_big7_x'] = df.loc[:,'d_big7'].div(df.loc[:,'dist_to_bc'])

In [ ]:
df['big8'] = df['s_bc']*df['s']
df['d_big8'] = abs(df.loc[:,'big8'].sub(df.loc[:,'big8'].shift()))
df.loc[df['nflId']!=df.loc[:,'nflId'].shift(), 'd_big8'] = 0
df['d_big8_x'] = df.loc[:,'d_big8'].div(df.loc[:,'dist_to_bc'])

In [ ]:
#df['big9'] = abs(df['a_x_bc']*df['a']*df['bc_x_perc']*df['frame_perc'])/df['dist_to_bc']
df['big9'] = abs(df['a_x_bc']*df['a'])/df['dist_to_bc']
#df['big9'] = abs(df['a_x_bc']*df['ds_to_bc'])/df['dist_to_bc']

In [ ]:
#df['big10'] = (df['a_bc']*df['a']*df['bc_x_perc']*df['frame_perc'])/df['dist_to_bc']
df['big10'] = abs(df['a_bc']*df['a'])/df['dist_to_bc']
#df['big10'] = abs(df['a_bc']*df['ds_to_bc'])/df['dist_to_bc']

In [ ]:
df['big11'] = (df['a_bc']*df['a']*df['dir_diff_perc'])/df['dist_to_bc']
df['big12'] = (df['a_bc']*df['a']*df['o_diff_perc'])/df['dist_to_bc']
df['big13'] = (df['a_bc']*df['a']*df['o_diff_perc']*df['dir_diff_perc'])/df['dist_to_bc']
df['big14'] = (df['a_bc']*df['a']*df['s_dir'])/df['dist_to_bc']

In [ ]:
param = 'big10'
dff = df.groupby(['gameId','playId','displayName','position','jerseyNumber'])[param].sum().reset_index(name='big_tot')
dff['big_tot_sum'] = dff.groupby(['gameId','playId'])['big_tot'].transform('sum')
dff['big_tot_min'] = dff.groupby(['gameId','playId'])['big_tot'].transform('min')
dff['big_tot_max'] = dff.groupby(['gameId','playId'])['big_tot'].transform('max')
dff = dff.merge(df.groupby(['gameId','playId','displayName','jerseyNumber'])[param].max().reset_index(name='big_max'))
dff['big_max_sum'] = dff.groupby(['gameId','playId'])['big_max'].transform('sum')
dff['big_max_min'] = dff.groupby(['gameId','playId'])['big_max'].transform('min')
dff['big_max_max'] = dff.groupby(['gameId','playId'])['big_max'].transform('max')
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['tackle'].max().reset_index(name='tackle'))
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['assist'].max().reset_index(name='assist'))
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['both'].max().reset_index(name='both'))
dff['jerseyNumber'] = dff['jerseyNumber'].astype('Int64')
dff['big_perc'] = (dff['big_tot']-dff['big_tot_min'])/(dff['big_tot_max']-dff['big_tot_min'])
dff['big_max_perc'] = (dff['big_max']-dff['big_max_min'])/(dff['big_max_max']-dff['big_max_min'])
dff['all_big'] = (dff['big_perc']+dff['big_max_perc'])/2
dff['big_perc_cum'] = np.round(dff['big_tot']/dff['big_tot_sum'],3)
dff['big_max_perc_cum'] = np.round(dff['big_max']/dff['big_max_sum'],3)
dff['all_big_cum'] = (dff['big_perc_cum']+dff['big_max_perc_cum'])/2
dff.reset_index(drop=True,inplace=True)
dff.head()

In [ ]:
min_plays = 100
min_players = dff.groupby('displayName')['playId'].count()[lambda x: x > min_plays].keys().tolist()
dff_min = dff[dff['displayName'].isin(min_players)].copy()

In [ ]:
dff_min['play_count'] = dff.groupby('displayName')['playId'].transform('size')
dff_min.head()

In [ ]:
min_big_perc = 0.25
dff_min['tackle_prob'] = 0
dff_min.loc[dff_min['all_big_cum']>min_big_perc,'tackle_prob'] = 1

In [ ]:
dff_.play_count

In [ ]:
dff_ = pd.DataFrame()
dff_['big_final'] = dff_min.groupby(['displayName','position'])['all_big_cum'].sum()
dff_['play_count'] = dff_min.groupby(['displayName','position'])['play_count'].max()
dff_['big_final_avg'] = dff_['big_final']/dff_['play_count']
dff_['big_final_q'] = dff_['big_final'].rank(pct=True)
dff_['both_final'] = dff_min.groupby(['displayName','position'])['both'].sum()
dff_['both_final_avg'] = dff_['both_final']/dff_['play_count']
dff_['both_final_q'] = dff_['both_final'].rank(pct=True)
dff_['tackle_final'] = dff_min.groupby(['displayName','position'])['tackle'].sum()
dff_['tackle_final_q'] = dff_['tackle_final'].rank(pct=True)
dff_['assist_final'] = dff_min.groupby(['displayName','position'])['assist'].sum()
dff_['assist_final_q'] = dff_['assist_final'].rank(pct=True)
dff_ = dff_.reset_index()
dff_.head()

In [ ]:
dff_.sort_values(by='big_final_avg',ascending=False)

In [ ]:
dff_[['big_final','both_final']].corr().iloc[0,1]

In [ ]:
dff_.position.unique()

In [ ]:
pos1 = ['DT','DE','NT']
pos2 = ['OLB','ILB','MLB']
pos3 = ['CB','FS','SS','DB']
dff_[dff_['position'].isin(pos3)].sort_values(by='big_final_avg',ascending=False).head(22)

In [ ]:
pos1 = ['DT','DE','NT']
pos2 = ['OLB','ILB','MLB']
pos3 = ['CB','FS','SS','DB']
dff_[dff_['position'].isin(pos1)].sort_values(by='big_final',ascending=False).head(22)

In [ ]:
dff_['diff'] = dff_['big_final_q'] - dff_['both_final_q']
dff_[dff_['both_final']>=20].sort_values(by='diff',ascending=False).head(33)

In [ ]:
_df = df.sample(1)
gid = _df['gameId'].values[0]
pid = _df['playId'].values[0]
print([gid, pid])

In [ ]:
gid = 2022101300
pid = 1000

In [ ]:
# subset the random play
df_plt = df.loc[(df['gameId']==gid) & (df['playId']==pid)].copy()

# set a threshold for how many players to plot at a time. I set it to the top 5 defenders who accrue the most tackle probability
#thresh = df_plt.loc[:, ['nflId', 'y_pred']].groupby('nflId', as_index=False).sum()
#thresh = thresh.sort_values('y_pred', ascending=False).head(6)['nflId'].unique()
#df_plt = df_plt.loc[df_plt['nflId'].isin(thresh)]


# i make a somewhat hacky highlighting of the player who makes the tackle by just plotting a bigger black line behind the "main" plot
#sns.lineplot(data=df_plt, x='frameId', y='y_pred', hue='tackle', palette=[(1,1,1,0),(0,0,0,1)], errorbar=('ci',False), lw=5, legend=False)

# this is the "main" plot, showcasing up to 5 defenders
#sns.lineplot(data=df_plt, x='frameId', y='y_pred', hue='displayName', palette='Set1')

#sns.lineplot(data=df_plt, x='frameId', y='dist_to_bc', hue='displayName', palette='Set2')
#sns.lineplot(data=df_plt, x='frameId', y='a_x_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='a', hue='displayName', palette='Set3')
#ns.lineplot(data=df_plt, x='frameId', y='big2', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='dir_diff_perc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='o_diff_perc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='da_x_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='ds_x_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='s_x_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='s_to_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='ds_to_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='a_bc', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='frame_perc', hue='displayName', palette='Set3')

sns.lineplot(data=df_plt, x='frameId', y='big10', hue='displayName', palette='Set3')
#sns.lineplot(data=df_plt, x='frameId', y='d_big7_x', hue='displayName', palette='Set3')

# removes some graph borders for a little cleaner look
sns.despine()

# moves the legend outside of the visualization "canvas", as to not obscure or block any of the visualization
plt.legend(bbox_to_anchor=(1.02,1), loc=2)

plt.axvline(x=df_plt[df_plt['event']=='first_contact']['frameId'].values[0],color='blue')
plt.axvline(x=df_plt[df_plt['event']=='tackle']['frameId'].values[0],color='red')
# print
plt.show()

In [ ]:
print('Quarter',plays.loc[(plays['gameId']==gid) & (plays['playId']==pid), 'quarter'].values[0])
print(plays.loc[(plays['gameId']==gid) & (plays['playId']==pid), 'playDescription'].values[0])
print('Teams:',df.loc[(df['gameId']==gid) & (df['playId']==pid),'possessionTeam'].values[0], 'vs',
               df.loc[(df['gameId']==gid) & (df['playId']==pid),'defensiveTeam'].values[0])
if df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['tackle']==1),'nflId'].values.size > 0:
    tackler = df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['tackle']==1),'nflId'].values[0]
    print('Tackler:',df.loc[df['nflId']==tackler,'displayName'].values[0], '#' + str(int(df.loc[df['nflId']==tackler,'jerseyNumber'].values[0])))

if df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['assist']==1),'nflId'].drop_duplicates().values.size > 0:
    assisters = df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['assist']==1),'nflId'].drop_duplicates().values
    for assister in assisters:
        print('Assister:',df.loc[df['nflId']==assister,'displayName'].values[0], '#' + str(int(df.loc[df['nflId']==assister,'jerseyNumber'].values[0])))

if df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['pff_missedTackle']==1),'nflId'].drop_duplicates().values.size > 0:
    missers = df.loc[(df['gameId']==gid) & (df['playId']==pid) & (df['pff_missedTackle']==1),'nflId'].drop_duplicates().values
    for misser in missers:
        print('Misser:',df.loc[df['nflId']==misser,'displayName'].values[0], '#' + str(int(df.loc[df['nflId']==misser,'jerseyNumber'].values[0])))

In [ ]:
df_plt = df.loc[(df['gameId']==gid) & (df['playId']==pid)].copy()

In [ ]:
temp = df_plt.groupby(['displayName','jerseyNumber'])['d_big7_x'].sum().reset_index(name='big_tot')
temp = temp.merge(df_plt.groupby(['displayName','jerseyNumber'])['d_big7_x'].max().reset_index(name='big_max'))
temp['jerseyNumber'] = temp['jerseyNumber'].astype('Int64')
temp['big_perc'] = (temp['big_tot']-temp['big_tot'].min())/(temp['big_tot'].max()-temp['big_tot'].min())
temp['big_max_perc'] = (temp['big_max']-temp['big_max'].min())/(temp['big_max'].max()-temp['big_max'].min())
temp['all_big'] = (temp['big_perc']+temp['big_max_perc'])/2
temp['big_perc_cum'] = np.round(temp['big_tot']/temp['big_tot'].sum(),3)
temp['big_max_perc_cum'] = np.round(temp['big_max']/temp['big_max'].sum(),3)
temp['all_big_cum'] = (temp['big_perc_cum']+temp['big_max_perc_cum'])/2
temp.sort_values(by='all_big',ascending=False).reset_index(drop=False)

In [ ]:
temp = df_plt.groupby(['displayName','jerseyNumber'])['d_big8_x'].sum().reset_index(name='big_tot')
temp = temp.merge(df_plt.groupby(['displayName','jerseyNumber'])['d_big8_x'].max().reset_index(name='big_max'))
temp['jerseyNumber'] = temp['jerseyNumber'].astype('Int64')
temp['big_perc'] = (temp['big_tot']-temp['big_tot'].min())/(temp['big_tot'].max()-temp['big_tot'].min())
temp['big_max_perc'] = (temp['big_max']-temp['big_max'].min())/(temp['big_max'].max()-temp['big_max'].min())
temp['all_big'] = (temp['big_perc']+temp['big_max_perc'])/2
temp['big_perc_cum'] = np.round(temp['big_tot']/temp['big_tot'].sum(),3)
temp['big_max_perc_cum'] = np.round(temp['big_max']/temp['big_max'].sum(),3)
temp['all_big_cum'] = (temp['big_perc_cum']+temp['big_max_perc_cum'])/2
temp.sort_values(by='all_big',ascending=False).reset_index(drop=False)

In [ ]:
temp = df_plt.groupby(['displayName','jerseyNumber'])['big9'].sum().reset_index(name='big_tot')
temp = temp.merge(df_plt.groupby(['displayName','jerseyNumber'])['big9'].max().reset_index(name='big_max'))
temp['jerseyNumber'] = temp['jerseyNumber'].astype('Int64')
temp['big_perc'] = (temp['big_tot']-temp['big_tot'].min())/(temp['big_tot'].max()-temp['big_tot'].min())
temp['big_max_perc'] = (temp['big_max']-temp['big_max'].min())/(temp['big_max'].max()-temp['big_max'].min())
temp['all_big'] = (temp['big_perc']+temp['big_max_perc'])/2
temp['big_perc_cum'] = np.round(temp['big_tot']/temp['big_tot'].sum(),3)
temp['big_max_perc_cum'] = np.round(temp['big_max']/temp['big_max'].sum(),3)
temp['all_big_cum'] = (temp['big_perc_cum']+temp['big_max_perc_cum'])/2
temp.sort_values(by='all_big',ascending=False).reset_index(drop=False)

In [ ]:
temp = df_plt.groupby(['displayName','jerseyNumber'])['big10'].sum().reset_index(name='big_tot')
temp = temp.merge(df_plt.groupby(['displayName','jerseyNumber'])['big10'].max().reset_index(name='big_max'))
temp['jerseyNumber'] = temp['jerseyNumber'].astype('Int64')
temp['big_perc'] = (temp['big_tot']-temp['big_tot'].min())/(temp['big_tot'].max()-temp['big_tot'].min())
temp['big_max_perc'] = (temp['big_max']-temp['big_max'].min())/(temp['big_max'].max()-temp['big_max'].min())
temp['all_big'] = (temp['big_perc']+temp['big_max_perc'])/2
temp['big_perc_cum'] = np.round(temp['big_tot']/temp['big_tot'].sum(),3)
temp['big_max_perc_cum'] = np.round(temp['big_max']/temp['big_max'].sum(),3)
temp['all_big_cum'] = (temp['big_perc_cum']+temp['big_max_perc_cum'])/2
temp.sort_values(by='all_big',ascending=False).reset_index(drop=False)

In [ ]:
test_final.groupby(['gameId','playId','nflId'])['both'].max()

In [ ]:
test_comp = pd.DataFrame()
test_comp['y_play'] = test_final.groupby(['gameId','playId','displayName'])['y_pred'].sum()
test_comp['y_play_sum'] = test_final.groupby(['gameId','playId'])['y_pred'].sum()
test_comp['y_perc'] = test_comp['y_play']/test_comp['y_play_sum']
test_comp['both_play'] = test_final.groupby(['gameId','playId','displayName'])['both'].max()

In [ ]:
plt.hist(test_comp[test_comp['both_play']==1]['y_play'], bins=100)
plt.show()

In [ ]:
plt.hist([test_comp[test_comp['both_play']==0]['y_perc'],test_comp[test_comp['both_play']==1]['y_perc']], \
         color=['Black', 'Red'], label=['False', 'True'], bins=100)
plt.show()

In [ ]:
test_comp[test_comp['both_play']==0].sort_values(by='y_perc',ascending=False).head(22)

In [ ]:
test_diff = test_comp.groupby('displayName').agg(y_perc_tot=('y_perc','sum'),both_play_tot=('both_play','sum'))
test_diff['y_q'] = test_diff['y_perc_tot'].rank(pct=True)
test_diff['both_q'] = test_diff['both_play_tot'].rank(pct=True)
test_diff['diff_q'] = test_diff['y_q'] - test_diff['both_q']

In [ ]:
test_diff[test_diff['both_play_tot']>=5].sort_values(by='diff_q',ascending=False)


In [ ]:
test_diff.corr().iloc[0,1]

In [ ]:
s = pd.Series()
p = {
      'objective':'binary:logistic',
      'eta':0.76,
      'reg_lambda':2.12,
      'scale_pos_weight':0.97,
    }
model = xgb.train(p,x_train)
_s = pd.Series(model.predict(x_test),index=test.index)
s = pd.concat([s,_s])

In [ ]:
model_data[f'{target}_xgb'] = s
model_data = model_data.merge(df.loc[:, ['gameId','playId','frameId','nflId','event', 'both']].drop_duplicates(), how='left')

In [ ]:
model_data.loc[model_data['both']==0, [target, f'{target}_xgb']].describe().round(3)

In [ ]:
model_data.loc[model_data['both']==1, [target, f'{target}_xgb']].describe().round(3)

In [ ]:
from xgboost import plot_importance

plot_importance(model)
plt.show()

In [ ]:
model_data[model_data['gameId'].isin(games_wk)].sample(1)

In [ ]:
_df = model_data[model_data['gameId'].isin(games_wk)].sample(1)
gid = _df['gameId'].values[0]
pid = _df['playId'].values[0]
print([gid, pid])

In [ ]:
# subset the random play
df_plt = model_data.loc[(model_data['gameId']==gid) & (model_data['playId']==pid)].copy()

# set a threshold for how many players to plot at a time. I set it to the top 5 defenders who accrue the most tackle probability
thresh = df_plt.loc[:, ['nflId', f'{target}_xgb']].groupby('nflId', as_index=False).sum()
thresh = thresh.sort_values(f'{target}_xgb', ascending=False).head(5)['nflId'].unique()
df_plt = df_plt.loc[df_plt['nflId'].isin(thresh)]

# i make a somewhat hacky highlighting of the player who makes the tackle by just plotting a bigger black line behind the "main" plot
sns.lineplot(data=df_plt, x='frameId', y=f'{target}_xgb', hue='both', palette=[(1,1,1,0),(0,0,0,1)], errorbar=('ci',False), lw=5, legend=False)

# this is the "main" plot, showcasing up to 5 defenders
sns.lineplot(data=df_plt, x='frameId', y=f'{target}_xgb', hue='displayName', palette='Set1')

# removes some graph borders for a little cleaner look
sns.despine()

# moves the legend outside of the visualization "canvas", as to not obscure or block any of the visualization
plt.legend(bbox_to_anchor=(1.02,1), loc=2)

# print
plt.show()

In [ ]:
plays.loc[(plays['gameId']==gid) & (plays['playId']==pid), 'playDescription'].values[0]

In [ ]:
df_plt.groupby('displayName')['both_xgb'].sum().sort_values(ascending=False)

In [ ]:
plays.loc[(plays['gameId']==gid) & (plays['playId']==pid), 'quarter'].values[0]

In [ ]:
df = pd.DataFrame()
current_game = run_plays_adj[0][0]

for pid in tqdm(run_plays_adj):
    print(pid)
    # subset data down to one play
    if current_game != pid[0]:
        df.reset_index(drop=True,inplace=True)
        df.to_csv(f'{project_dir}{current_game}.csv',index=False)
        df = pd.DataFrame()
        current_game = pid[0]

    play = week.loc[(week['gameId']==pid[0])&(week['playId']==pid[1])].copy()
    for fid in play['frameId'].unique():
        # subset data down to one frame
        frame = play.loc[play['frameId']==fid].copy()

        # make unique positions, as to not duplicate columns based on player position
        frame['pos_unique'] = (frame['position']
                            .add(frame
                                    .groupby('position', as_index=False)
                                    .cumcount()
                                    .add(1)
                                    .dropna()
                                    .astype(str)
                                    .str.replace('.0','', regex=False)
                                    .str.replace('0','', regex=False)))

        # calc distances
        _df = (pd
            .DataFrame(cdist(frame.loc[:, ['x', 'y']],
                                frame.loc[:, ['x', 'y']]),
                        index=frame['nflId'],
                        columns=frame['pos_unique'].fillna('football')))

        # reset index to pop out nflId into its own column
        _df = _df.reset_index()

        # merge new distance values onto original dataframe
        frame = frame.merge(_df)
        frame['orient_modifier'] = 0.0
        for name in frame.displayName.unique():
            if name != 'football':
                y_diff = frame.loc[frame['displayName']=='football','y'].values[0] - frame.loc[frame['displayName']==name,'y'].values[0]
                x_diff = frame.loc[frame['displayName']=='football','x'].values[0] - frame.loc[frame['displayName']==name,'x'].values[0]
                slope = x_diff/max(0.1,y_diff)
                orient_to_football = math.degrees(np.arctan(slope))
                orient_diff = frame.loc[frame['displayName']==name,'o'].values[0] - orient_to_football
                orient_diff = abs((orient_diff + 180) % 360 - 180)
                frame.loc[frame['displayName']==name,'orient_diff'] = (180-orient_diff)/180
                #frame.loc[frame['displayName']==name,'orient_diff'] = orient_diff_perc

                dir_diff = frame.loc[frame['displayName']==name,'dir'].values[0] - orient_to_football
                dir_diff = abs((dir_diff + 180) % 360 - 180)
                frame.loc[frame['displayName']==name,'dir_diff'] = (180-dir_diff)/180
                #frame.loc[frame['displayName']==name,'dir_diff'] = dir_diff_perc
                #frame.loc[frame['displayName']==name,'orient_modifier'] = 0.5*orient_diff_perc + 0.5

        # concatenate new results into the output dataframe
        df = pd.concat([df,frame])


df.reset_index(drop=True,inplace=True)
df.to_csv(f'{project_dir}{current_game}.csv',index=False)
send_message('7202725275','att','biscuits are done!')

In [ ]:
run_plays_adj

In [ ]:
df.ballCarrierId

In [ ]:
plays.columns

In [ ]:
df.shape

In [ ]:
pd.merge(df, plays,  how='left', left_on=['gameId','playId'], right_on = ['gameId','playId'])

In [ ]:
run_plays_adj[0]

In [ ]:
df = df_.copy()

In [ ]:
run_plays_x = run_plays_adj[0]

In [ ]:

test = df[(df['gameId']==run_plays_x[0])&(df['playId']==run_plays_x[1])].copy()
test = pd.merge(test, plays,  how='left', left_on=['gameId','playId'], right_on = ['gameId','playId'])
df_play = calibrate_coords(test)
nflid = df_play['ballCarrierId'].unique()[0]
bc = df_play.copy()
bc = bc[bc['nflId']==nflid]
bc['x_bc'] = bc['x']
bc['y_bc'] = bc['y']
bc['s_bc'] = bc['s']
bc['dir_bc'] = bc['dir']
bc = bc[['gameId','playId','frameId','x_bc','y_bc','s_bc','dir_bc']]
df_play.reset_index(drop=True,inplace=True)
bc.reset_index(drop=True,inplace=True)
df_play = pd.merge(df_play, bc,  how='left', left_on=['gameId','playId','frameId'], right_on = ['gameId','playId','frameId'])
df_play = df_play[(df_play['displayName']!='football')&(df_play['club']==df_play['defensiveTeam'])]

In [ ]:
df_play.reset_index(drop=True,inplace=True)
df_play['opt_dist_diff'] = 0.0
df_play['opt_dir_diff'] = 0.0
df_play['opt_dir_diff_perc'] = 0.0
df_play['dist_total'] = 0.0
for i in range(len(df_play)):
    df_play.loc[i,'dist_total'] += df_play.loc[i,'dis']
    d_vect = {'x': df_play.loc[i,'x'],
              'y': df_play.loc[i,'y'],
              's': df_play.loc[i,'s'],
              'dir': np.radians(df_play.loc[i,'dir'])
              }
    bc_vect = {'x': df_play.loc[i,'x_bc'],
              'y': df_play.loc[i,'y_bc'],
              's': df_play.loc[i,'s_bc'],
              'dir': np.radians(df_play.loc[i,'dir_bc'])
              }
    opt_vect = calc_opt_intersection(d_vect,bc_vect)
    if opt_vect['noop']==True:
        df_play.loc[i,'opt_dist_diff'] = np.nan
        df_play.loc[i,'opt_dir_diff'] = np.nan
    else:
        diff_theta = abs(np.radians(df_play.loc[i,'dir'])-opt_vect['dir'])
        diff_theta = abs((diff_theta + np.pi) % (2*np.pi) - np.pi)
        diff_theta_perc = (np.pi-diff_theta)/np.pi

        df_play.loc[i,'opt_dist_diff'] = 2*df_play.loc[i,'dis']*np.sin(diff_theta/2)
        df_play.loc[i,'opt_dir_diff'] = diff_theta
        df_play.loc[i,'opt_dir_diff_perc'] = diff_theta_perc

In [ ]:
df_play.head(33)

In [ ]:
gameIds, playIds = zip(*run_plays_adj)

In [ ]:
type(list(set(gameIds)))

In [ ]:
gameIds, playIds = zip(*run_plays_adj)
df = pd.DataFrame()
for game in tqdm(set(gameIds),leave=False):
    temp = pd.read_csv(f'{project_dir}{game}prep.csv')
    temp = temp.sort_values(['gameId','playId','nflId','frameId']).reset_index(drop=True)
    df = pd.concat([df,temp])
#df.to_csv(f'{project_dir}games_processed_3.csv',mode='a',index=False)

In [ ]:
df.to_csv(f'{project_dir}games_processed_3.csv',mode='a',index=False)

In [ ]:
df = df_.copy()

In [ ]:
current_game = run_plays_adj[0][0]
X_prep = pd.DataFrame()
for pid in tqdm(run_plays_adj,leave=False):
    if current_game != pid[0]:
        print('-------- NEW GAME --------')
        print(pid,'feature_gen')
        X_prep.reset_index(drop=True,inplace=True)
        X_prep.to_csv(f'{project_dir}{current_game}prep.csv',index=False)
        X_prep = pd.DataFrame()
        current_game = pid[0]
    else:
        print(pid,'feature_gen')
    df_pid = df[(df['gameId']==pid[0])&(df['playId']==pid[1])].copy()
    X_temp = feature_gen_4(df_pid, plays)

    if X_prep.empty:
        X_prep = X_temp
    else:
        X_prep = pd.concat([X_prep,X_temp])

X_prep.reset_index(drop=True,inplace=True)
X_prep.to_csv(f'{project_dir}{current_game}prep.csv',index=False)
send_message('7202725275','att','biscuits are done!')

In [ ]:
X_prep.to_csv(f'{project_dir}{current_game}prep.csv',index=False)

In [ ]:
gameIds, playIds = zip(*run_plays_adj)
gameIds = pd.DataFrame(list(gameIds),columns=['games'])
gameIds.games.unique()

In [ ]:
gid = 2022110610
X_prep = pd.read_csv(f'{project_dir}{gid}prep.csv')
X_prep[(X_prep['gameId']==gid)]['playId'].unique()

In [ ]:
pid = 737
temp = X_prep[(X_prep['gameId']==gid)&(X_prep['playId']==pid)].copy()
temp.reset_index(drop=True,inplace=True)
temp['dir'] = np.radians(temp['dir'])
temp[temp['tackle']==1][['displayName','opt_dir_diff_perc','opt_x','opt_y','x','y','opt_dir','dir']]


In [ ]:
x = temp['x']
y = temp['y']

fig = plt.figure()
ax = plt.axes(xlim=(0,120), ylim=(0,54))
ax.plot(x, y, marker='o')

# initialization function
def init():
   pass

# animation function
def animate(i):
   x = temp['x']
   y = temp['y']
   ax.plot(x, y, marker='o')

# setting a title for the plot
plt.title(f'{gid}_{pid}')

anim = animation.FuncAnimation(fig, animate, init_func=init, interval=1)

plt.show()

In [ ]:
temp['x'].tolist()[:1][0]

In [ ]:
x = temp['x']
y = temp['y']
#size = np.random.randint(150, size=10)
colors = np.random.choice(["r", "g", "b"], size=10)

fig = plt.figure()
plt.xlim(0, 100)
plt.ylim(0, 55)
graph = plt.scatter([], [])

def animate(i):
    graph.set_offsets(np.vstack((x[:i+1], y[:i+1])).T)
    #graph.set_sizes(size[:i+1])
    graph.set_facecolors(colors[:i+1])
    return graph

ani = FuncAnimation(fig, animate, repeat=False, interval=200)
plt.show()

In [ ]:
params = ['BIG_total_perc','BIG_total','opt_x','opt_y','opt_dir','opt_dist_diff','opt_dir_diff','opt_dir_diff_perc']

X_prep = pd.DataFrame()
for pid in tqdm(run_plays_adj, leave=False):
    print('play',pid,'feature_gen')
    df_pid = df[(df['gameId']==pid[0])&(df['playId']==pid[1])]
    df_features = feature_gen_3(df_pid, plays)
    X_temp = train_test_gen_3(df_features)

    if X_prep.empty:
        X_prep = X_temp
    else:
        X_prep = pd.concat([X_prep,X_temp])

X_prep.reset_index(drop=True,inplace=True)
filename_id_X = datetime.now().strftime('X_%m%d%Y_%H_%M.csv')
X_prep.to_csv(f'{filename_id_X}',index=False)
send_message('7202725275','att','biscuits are done!')

In [ ]:
X_prep.shape

In [ ]:
X_prep.columns

In [ ]:
X_prep[X_prep['gameId']==gid]['playId'].unique()

In [ ]:
gid = 2022110700
pid = 167
temp = X_prep[(X_prep['gameId']==gid)&(X_prep['playId']==pid)].copy()
temp[temp['tackle']==1][['displayName','opt_dir_diff_perc','dist_total','opt_x','opt_y','opt_dir']]

In [ ]:
temp.displayName.unique()

In [ ]:
min_plays = 100
play_count = X_prep.groupby(['displayName'])['playId'].count()
over_list = play_count[play_count > min_plays].index.tolist()
X_prep = X_prep[X_prep['displayName'].isin(over_list)]

In [ ]:
x_total = pd.DataFrame()
x_total['big'] = X_prep.groupby(['displayName'])['BIG_total_perc'].sum()
x_total['big_avg'] = X_prep.groupby(['displayName'])['BIG_total_perc'].mean()
x_total['big_t'] = X_prep.groupby(['displayName'])['BIG_total'].sum()
x_total['big_avg_t'] = X_prep.groupby(['displayName'])['BIG_total'].mean()
x_total['tackle'] = X_prep.groupby(['displayName'])['tackle'].sum()
x_total['assist'] = X_prep.groupby(['displayName'])['assist'].sum()
x_total['missed'] = X_prep.groupby(['displayName'])['missed'].sum()
x_total['both'] = x_total['tackle'] + x_total['assist']
x_total.reset_index(inplace=True)
x_total = x_total.merge(X_prep[['displayName','position']],how='left',on='displayName').drop_duplicates(subset='displayName')
x_total = x_total.sort_values(by='big_t',ascending=False)

In [ ]:
X_prep.shape

In [ ]:
test = df[(df['gameId']==2022100907)&(df['playId']==2210)].copy()

In [ ]:
temp = X_prep.copy()

In [ ]:
min_plays = 100
play_count = temp.groupby(['displayName'])['playId'].count()
over_list = play_count[play_count > min_plays].index.tolist()
temp = temp[temp['displayName'].isin(over_list)]

In [ ]:
temp.shape

In [ ]:
temp['BIG_total'].clip(temp['BIG_total'].quantile(0.05),temp['BIG_total'].quantile(0.95),inplace=True)

In [ ]:
x_total = pd.DataFrame()
x_total['big_t'] = temp.groupby('displayName').apply(lambda x: x['BIG_total'].clip(x['BIG_total'].quantile(0.05),x['BIG_total'].quantile(0.95)).sum())
x_total['big_avg_t'] = temp.groupby('displayName').apply(lambda x: x['BIG_total'].clip(x['BIG_total'].quantile(0.05),x['BIG_total'].quantile(0.95)).mean())
x_total['tackle'] = temp.groupby(['displayName'])['tackle'].sum()
x_total['assist'] = temp.groupby(['displayName'])['assist'].sum()
x_total['missed'] = temp.groupby(['displayName'])['missed'].sum()
x_total['tackle_avg'] = temp.groupby(['displayName'])['tackle'].mean()
x_total['assist_avg'] = temp.groupby(['displayName'])['assist'].mean()
x_total['both'] = x_total['tackle'] + x_total['assist']
x_total['both_avg'] = x_total['tackle_avg'] + x_total['assist_avg']
x_total.reset_index(inplace=True)
x_total = x_total.merge(X_prep[['displayName','position']],how='left',on='displayName').drop_duplicates(subset='displayName')
x_total = x_total.sort_values(by='big_t',ascending=False)

In [ ]:
x_total[x_total['both']>20].sort_values(by='big_avg_t',ascending=False).head(22)

In [ ]:
x_total['big_q_t'] = x_total['big_t'].rank(pct=True)
x_total['big_avg_q_t'] = x_total['big_avg_t'].rank(pct=True)
x_total['both_q'] = x_total['both'].rank(pct=True)
x_total['both_avg_q'] = x_total['both_avg'].rank(pct=True)
x_total.head(33)

In [ ]:
print('big_a corr',(x_total[['big_t','both']].corr().iloc[0,1]))

In [ ]:
print('big_a corr',(x_total[['big_avg_t','both_avg']].corr().iloc[0,1]))

In [ ]:
x_total['diff'] = x_total['big_q_t'] - x_total['both_q']
x_total[x_total['both']>=30].sort_values(by='diff',ascending=False).head(33)

In [ ]:
x_total[x_total['both']>=30].sort_values(by='big_avg_t',ascending=False).head(22)

In [ ]:
X_prep['BIG_total'].median()

In [ ]:
X_prep.groupby('displayName')['BIG_total'].value_counts()

In [ ]:
X_prep['BIG_total'].hist(bins=100)

plt.hist(X_prep['BIG_total'], bins=100, color='c', edgecolor='k', alpha=0.65)
plt.axvline(X_prep['BIG_total'].median(), color='k', linestyle='dashed',linewidth=1)
plt.show()

In [ ]:
x_total.position.unique()

In [ ]:
x_total[x_total['displayName'].str.contains('Singleton')][['displayName','big_avg_t','big_avg_q']]

In [ ]:
pos_group = ['DT','NT','DE']
#pos_group = ['ILB','OLB','MLB']
#pos_group = ['FS','SS','CB','DB']
x_total[x_total['position'].isin(pos_group)&(x_total['both']>20)].drop_duplicates(subset='displayName').sort_values(by='big_avg_t',ascending=False).head(33)

In [ ]:
pos_group = ['DT','NT','DE']
#pos_group = ['ILB','OLB','MLB']
#pos_group = ['FS','SS','CB','DB']
x_total[(x_total['position'].isin(pos_group))&(x_total['both']>=20)].sort_values(by='diff',ascending=False).head(33)

In [ ]:
temp[(temp['displayName']=='Devin Lloyd')&(temp['playId']==3464)]

In [ ]:
#X_prep[X_prep['both']==1]['BIG_total'].mean()
#X_prep[X_prep['both']==0]['BIG_total'].mean()
#X_prep[X_prep['both']==1]['BIG_total'].median()
X_prep[X_prep['both']==0]['BIG_total'].median()

In [ ]:
X_prep[X_prep['both']==1]['BIG_total'].hist()
#X_prep[X_prep['both']==0]['BIG_total'].hist()
plt.show()

In [ ]:
X_prep[X_prep['both']==0].sort_values(by='BIG_total',ascending=False).head(33)

In [ ]:
gid = 2022100906
pid = 1739

print('Quarter',plays[(plays['gameId']==gid)&(plays['playId']==pid)]['quarter'].values[0])
print(plays[(plays['gameId']==gid)&(plays['playId']==pid)]['playDescription'].values[0])
tacklers = tackles[(tackles['gameId']==gid)&(tackles['playId']==pid)&(tackles['tackle']==1)]['nflId'].values
tacklers
for tackler in tacklers:
    print('Tackler:',df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==tackler)]['displayName'].drop_duplicates().values[0],'#' + \
                    str(int(df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==tackler)]['jerseyNumber'].drop_duplicates().values[0])))
assisters = tackles[(tackles['gameId']==gid)&(tackles['playId']==pid)&(tackles['assist']==1)]['nflId'].values
for assister in assisters:
    print('Assister:',df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==assister)]['displayName'].drop_duplicates().values[0],'#' + \
                    str(int(df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==assister)]['jerseyNumber'].drop_duplicates().values[0])))
missers = tackles[(tackles['gameId']==gid)&(tackles['playId']==pid)&(tackles['pff_missedTackle']==1)]['nflId'].values
for misser in missers:
    print('Misser:',df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==misser)]['displayName'].drop_duplicates().values[0],'#' + \
                    str(int(df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==misser)]['jerseyNumber'].drop_duplicates().values[0])))
#print(df[(df['gameId']==gid)&(df['playId']==pid)&(df['nflId']==id)]['displayName'].values)

In [ ]:
X_prep[(X_prep['gameId']==gid)&(X_prep['playId']==pid)][['displayName','jerseyNumber','both','BIG_total_perc']].sort_values(by='BIG_total_perc',ascending=False)

In [ ]:
X_prep[X_prep['displayName'].str.contains('Koramoah')][['gameId','playId','both','BIG_total_perc']].sort_values(by='BIG_total_perc',ascending=False).head(33)

In [ ]:
gid = 2022100900
pid = 3589

In [ ]:
df_pid = df[(df['gameId']==gid)&(df['playId']==pid)]
df_features = feature_gen_3(df_pid, plays)

In [ ]:
df_features[['displayName','BIG_total_perc']].drop_duplicates(subset='displayName').sort_values(by='BIG_total_perc',ascending=False)

In [ ]:
df_features.groupby('displayName')['BIG'].sum().sort_values(ascending=False)

In [ ]:
df_features[(df_features['gameId']==2022100906)&(df_features['playId']==1739)][['displayName','jerseyNumber','BIG_total']].drop_duplicates(subset='displayName').sort_values(by='BIG_total',ascending=False)

In [ ]:
#names = ['Jeremiah Owusu-Koramoah','Jordan Elliott']
#names = ['Khalil Mack']
#names = ['Alex Singleton']
#for name in names:
for name in df_features.displayName.unique():

    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['football'], label=name)
    plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['dir_modifier'], label=name)
    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['orient_diff'], label='orient')
    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['orient_modifier'], label=name)
    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['BIG'], label=name)
    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['s'], label=name)
    #plt.plot(df_features[df_features['displayName']==name]['frameId'],df_features[df_features['displayName']==name]['BIG_a'], label='big_a')


plt.plot(df_features['frameId'],df_features['dS_x'],label='dS_x')
#plt.plot(df_features['frameId'],df_features['dS_x_d'],label='dS_x_d')
#plt.plot(df_features['frameId'],df_features['dS_x_d_roll'],label='dS_x_d_roll')
#plt.plot(df_features['frameId'],df_features['dS_x_roll'],label='dS_x_roll')
#plt.plot(df_features['frameId'],df_features['dA_x'],label='dA_x')
#plt.plot(df_features['frameId'],df_features['s_x'],label='s_x')
#plt.plot(df_features['frameId'],df_features['a_x'],label='a_x')

for i in range(len(df_features)):
    if df_features['event'].iloc[i] == 'tackle':
        plt.axvline(x=df_features['frameId'].iloc[i], color='red')
    elif df_features['event'].iloc[i] == 'first_contact':
        plt.axvline(x=df_features['frameId'].iloc[i], color='blue')
    elif df_features['event'].iloc[i] == 'out_of_bounds':
        plt.axvline(x=df_features['frameId'].iloc[i], color='green')
    elif df_features['event'].iloc[i] == 'fumble':
        plt.axvline(x=df_features['frameId'].iloc[i], color='orange')

#lt.xlim(27, 39)
plt.legend(loc='lower left',fontsize=7)
plt.show()

In [ ]:
test = df[(df['gameId']==2022100907)&(df['playId']==2210)].copy()

In [ ]:
tester = feature_gen_3(test,plays)

In [ ]:
tester['s']

In [ ]:
tester['BIG_total']

In [ ]:
def feature_gen_3(df, plays):

    df_ = pd.merge(df, plays,  how='left', left_on=['gameId','playId'], right_on = ['gameId','playId'])
    nflid = df_['ballCarrierId'].unique()[0]
    df_play = calibrate_coords(df_)
    frame_min = df_play['frameId'].min()
    frame_max = df_play['frameId'].max()

    roll_frames = 1
    if np.any(df_play['event']=='first_contact'):
        #temp['first_contact_to_tackle_diff'] = frame_max - temp[temp['event']=='first_contact']['frameId'].values[0]
        roll_frames = frame_max - df_play[df_play['event']=='first_contact']['frameId'].values[0]

    first_contact_frame = frame_max - roll_frames
    fcf_halfway = round((frame_max - first_contact_frame)/2) + first_contact_frame

    bc = df_play.copy()
    bc = bc[bc['nflId']==nflid]
    bc['x_bc'] = bc['x']
    bc['y_bc'] = bc['y']
    bc['s_bc'] = bc['s']
    bc['dir_bc'] = bc['dir']
    x_total = bc['x'].max() - bc['x'].min()
    bc['x_perc'] = (bc['x'] - bc['x'].min())/x_total
    bc['d_x'] = bc.loc[:,'x'].sub(bc.loc[:,'x'].shift()).div(0.1).fillna(0)
    bc['s_x'] = np.sin(np.radians(bc['dir'])) * bc['s']
    bc['dS_x'] = bc.loc[:,'s_x'].sub(bc.loc[:,'s_x'].shift(-1)).fillna(0)
    bc['dS_x_d'] = bc.loc[:,'dS_x'].sub(bc.loc[:,'dS_x'].shift(-1)).fillna(0)
    bc['dS_x_roll'] = bc.loc[:,'dS_x'].rolling(window=2,min_periods=1).mean()
    bc['dS_x_d_roll'] = -bc.loc[:,'dS_x_d'].rolling(window=3,min_periods=1).mean()
    bc['a_x'] = np.sin(np.radians(bc['dir'])) * bc['a']
    bc['dA_x'] = bc.loc[:,'a_x'].sub(bc.loc[:,'a_x'].shift(-1)).fillna(0)

    #for i in range(len(bc)):
    #    idx = bc.index[i]
    #    if bc.loc[idx,'y'] > (160/3):
    #        bc.loc[idx,'y'] = (160/3)
    #    elif bc.loc[idx,'y'] < 0:
    #        bc.loc[idx,'y'] = 0

    #bc['y_adj'] = np.abs(bc['y'] - (160/6))
    #bc['d_y'] = bc.loc[:,'y_adj'].sub(bc.loc[:,'y_adj'].shift(-1)).div(0.1).fillna(0)
    #ep1 = 1
    #ep2 = 0
    #bc['boundary'] = np.exp(bc['y_adj']-ep1*(ep2+160/6)) * bc['d_y']
    #bc['dsx'] = bc['dS_x']
    bc = bc[['gameId','playId','frameId','s_x','dS_x','dA_x','x_perc','a_x','dS_x_roll','dS_x_d','dS_x_d_roll','x_bc','y_bc','s_bc','dir_bc']]
    df_play = pd.merge(df_play, bc,  how='left', left_on=['gameId','playId','frameId'], right_on = ['gameId','playId','frameId'])
    #set to Defense
    df_play = df_play[df_play['club']==df_play['defensiveTeam']]

    #df_play['activation'] = -(1/3)*df_play['football']+1   # linearly increases from zero to 1 once defender within 3 yards of football
    df_play['zero'] = 0
    df_play['football_min_dist'] = 0.33
    #df_play['boundary_final'] = df_play[['activation','zero']].values.max(1) * df_play['boundary']
    df_play['football'] = df_play[['football','football_min_dist']].values.max(1)
    #df_play['orient_modifier'] = 0.5*df_play['orient_diff'] + 0.5
    #df_play['orient_modifier'] = np.sqrt(df_play['orient_diff'])
    #df_play['orient_modifier'] = (1/3)*df_play['orient_diff'] + (2/3)
    #df_play['orient_modifier'] = np.sqrt(df_play['orient_diff'])
    #df_play['orient_modifier'] = df_play['orient_diff'].rolling(window=10,min_periods=1).mean()
    df_play['dir_diff'] = df_play['dir_diff'].fillna(0)
    df_play['s'] = df_play['s'].fillna(0)
    df_play.reset_index(drop=True,inplace=True)
    df_play.loc[df_play['dir_diff']<0.5,'dir_modifier'] = 0
    df_play.loc[df_play['dir_diff']>=0.5,'dir_modifier'] = 2*(df_play['dir_diff']-0.5)*df_play['s']
    #df_play['BIG'] = df_play['dS_x'].div(df_play['football'].shift(-1)) + df_play['boundary_final']
    #df_play['BIG_x'] = df_play['dS_x'].div(df_play['football'].shift(-1))*df_play['x_perc']*df_play['orient_modifier'] + df_play['boundary_final']
    df_play['x_perc'] = df_play['x_perc'].fillna(0)
    #df_play['dir_diff'] = (df_play['dir_diff'].fillna(0))*(df_play['s'].fillna(0))
    #df_play['orient_diff'] = df_play['orient_diff'].fillna(0)
    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG'] = df_play['dS_x'].div(df_play['football'].shift(-1))*df_play['x_perc'].shift(-1)*df_play['dir_modifier'].shift(-1)
    #df_play['BIG'] = df_play[['zero','BIG']].values.max(1)
    #df_play['BIG_a'] = df_play['dA_x'].div(df_play['football'].shift(-2))*df_play['x_perc']*df_play['orient_diff'].shift(-2) + df_play['boundary_final']
    #df_play['BIG_b'] = df_play['dS_x'].div(df_play['football'].shift(-1))*df_play['x_perc']*np.sqrt(df_play['orient_diff']) + df_play['boundary_final']
    #df_play['BIG_a'] = df_play['dA_x'].div(df_play['football'].shift(-1))*df_play['x_perc']*df_play['orient_modifier']


    #df_play['BIG_total'] = df_play.groupby('nflId')['BIG'].transform('sum')
    #df_play['BIG_total_sum'] = df_play.groupby('frameId')['BIG_total'].transform('sum')
    #df_play['BIG_total_perc'] = np.round(df_play['BIG_total']/df_play['BIG_total_sum'],5)
    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_total'] = df_play.groupby('nflId')['BIG'].transform('sum')
    #if df_play.groupby('frameId')['BIG_total'].transform('min')[0] < 0:
    #    BIG_total_min = abs(df_play.groupby('frameId')['BIG_total'].transform('min')[0])
    #else:
    #    BIG_total_min = 0
    #df_play['BIG_total_adj'] = df_play['BIG_total'] + BIG_total_min
    df_play['BIG_total_sum'] = df_play.groupby('frameId')['BIG_total'].transform('sum')
    df_play['BIG_total_perc'] = df_play['BIG_total']/df_play['BIG_total_sum']


    #df_play.reset_index(drop=True,inplace=True)
    #grouped = df_play[df_play['frameId'].between(frame_min,frame_max-5)].groupby('nflId')
    #for name, group in grouped:
    #    df_play.loc[df_play['nflId']==int(name),'BIG_total_shift'] = group['BIG'].sum()

    #df_play.reset_index(drop=True,inplace=True)
    #if df_play.groupby('frameId')['BIG_total_shift'].transform('min')[0] < 0:
    #    BIG_total_min = abs(df_play.groupby('frameId')['BIG_total_shift'].transform('min')[0])
    #else:
    #    BIG_total_min = 0
    #df_play['BIG_total_adj_shift'] = df_play['BIG_total_shift'] + BIG_total_min
    #df_play['BIG_total_sum_shift'] = df_play.groupby('frameId')['BIG_total_adj_shift'].transform('sum')
    #df_play['BIG_total_perc_shift'] = df_play['BIG_total_adj_shift']/df_play['BIG_total_sum_shift']

    #-------------------------------


    #df_play.reset_index(drop=True,inplace=True)
    #grouped = df_play[df_play['frameId'].between(frame_min,frame_max-5)].groupby('nflId')
    #for name, group in grouped:
    #    df_play.loc[df_play['nflId']==int(name),'BIG_total_shift_a'] = group['BIG_a'].sum()

    #df_play.reset_index(drop=True,inplace=True)
    #if df_play.groupby('frameId')['BIG_total_shift_a'].transform('min')[0] < 0:
    #    BIG_total_min = abs(df_play.groupby('frameId')['BIG_total_shift_a'].transform('min')[0])
    #else:
    #    BIG_total_min = 0
    #df_play['BIG_total_adj_shift_a'] = df_play['BIG_total_shift_a'] + BIG_total_min
    #df_play['BIG_total_sum_shift_a'] = df_play.groupby('frameId')['BIG_total_adj_shift_a'].transform('sum')
    #df_play['BIG_total_perc_shift_a'] = df_play['BIG_total_adj_shift_a']/df_play['BIG_total_sum_shift_a']

    df_play.reset_index(drop=True,inplace=True)
    df_play['opt_dist_diff'] = 0.0
    df_play['opt_dir_diff'] = 0.0
    df_play['opt_dir_diff_perc'] = 0.0
    df_play['dist_total'] = 0.0
    for i in range(len(df_play)):
        df_play.loc[i,'dist_total'] += df_play.loc[i,'dis']
        d_vect = {'x': df_play.loc[i,'x'],
                'y': df_play.loc[i,'y'],
                's': df_play.loc[i,'s'],
                'dir': np.radians(df_play.loc[i,'dir'])
                }
        bc_vect = {'x': df_play.loc[i,'x_bc'],
                'y': df_play.loc[i,'y_bc'],
                's': df_play.loc[i,'s_bc'],
                'dir': np.radians(df_play.loc[i,'dir_bc'])
                }
        opt_vect = calc_opt_intersection(d_vect,bc_vect)
        if opt_vect['noop']==True:
            df_play.loc[i,'opt_x'] = np.nan
            df_play.loc[i,'opt_y'] = np.nan
            df_play.loc[i,'opt_dir'] = np.nan
            df_play.loc[i,'opt_dist_diff'] = np.nan
            df_play.loc[i,'opt_dir_diff'] = np.nan
        else:
            diff_theta = abs(np.radians(df_play.loc[i,'dir'])-opt_vect['dir'])
            diff_theta = abs((diff_theta + np.pi) % (2*np.pi) - np.pi)
            diff_theta_perc = (np.pi-diff_theta)/np.pi

            df_play.loc[i,'opt_x'] = opt_vect['x']
            df_play.loc[i,'opt_y'] = opt_vect['y']
            df_play.loc[i,'opt_dir'] = opt_vect['dir']
            df_play.loc[i,'opt_dist_diff'] = 2*df_play.loc[i,'dis']*np.sin(diff_theta/2)
            df_play.loc[i,'opt_dir_diff'] = diff_theta
            df_play.loc[i,'opt_dir_diff_perc'] = diff_theta_perc

    return df_play

In [ ]:
def feature_gen_4(df, plays):

    df_ = pd.merge(df, plays,  how='left', left_on=['gameId','playId'], right_on = ['gameId','playId'])
    nflid = df_['ballCarrierId'].unique()[0]
    df_play = calibrate_coords(df_)


    bc = df_play.copy()
    bc = bc[bc['nflId']==nflid]
    bc['x_bc'] = bc['x']
    bc['y_bc'] = bc['y']
    bc['s_bc'] = bc['s']
    bc['dir_bc'] = bc['dir']

    bc = bc[['gameId','playId','frameId','x_bc','y_bc','s_bc','dir_bc']]
    df_play = pd.merge(df_play, bc,  how='left', left_on=['gameId','playId','frameId'], right_on = ['gameId','playId','frameId'])
    #set to Defense
    df_play = df_play[(df_play['club']==df_play['defensiveTeam'])&(df_play['displayName']!='football')]

    df_play.reset_index(drop=True,inplace=True)
    df_play['opt_dist_diff'] = 0.0
    df_play['opt_dir_diff'] = 0.0
    df_play['opt_dir_diff_perc'] = 0.0
    df_play['dist_total'] = 0.0
    df_prep = pd.DataFrame()
    for name in df_play['displayName'].unique():
        temp = df_play[df_play['displayName']==name].copy()
        temp.reset_index(drop=True,inplace=True)
        a_max = 0.1*temp['a'].max()
        s_max = temp['s'].max()
        for i in range(len(temp)):
            temp.loc[i,'dist_total'] += temp.loc[i,'dis']
            d_vect = {'x': temp.loc[i,'x'],
                    'y': temp.loc[i,'y'],
                    's': min((temp.loc[i,'s'] + a_max),s_max),
                    'dir': np.radians(temp.loc[i,'dir'])
                    }
            bc_vect = {'x': temp.loc[i,'x_bc'],
                    'y': temp.loc[i,'y_bc'],
                    's': temp.loc[i,'s_bc'],
                    'dir': np.radians(temp.loc[i,'dir_bc'])
                    }
            opt_vect = calc_opt_intersection(d_vect,bc_vect)
            if opt_vect['noop']==True:
                temp.loc[i,'opt_x'] = np.nan
                temp.loc[i,'opt_y'] = np.nan
                temp.loc[i,'opt_dir'] = np.nan
                temp.loc[i,'opt_dist_diff'] = np.nan
                temp.loc[i,'opt_dir_diff'] = np.nan
            else:
                diff_theta = abs(np.radians(temp.loc[i,'dir'])-opt_vect['dir'])
                diff_theta = abs((diff_theta + np.pi) % (2*np.pi) - np.pi)
                diff_theta_perc = (np.pi-diff_theta)/np.pi

                temp.loc[i,'opt_x'] = opt_vect['x']
                temp.loc[i,'opt_y'] = opt_vect['y']
                temp.loc[i,'opt_dir'] = opt_vect['dir']
                temp.loc[i,'opt_dist_diff'] = 2*temp.loc[i,'dis']*np.sin(diff_theta/2)
                temp.loc[i,'opt_dir_diff'] = diff_theta
                temp.loc[i,'opt_dir_diff_perc'] = diff_theta_perc
        df_prep = pd.concat([df_prep,temp])

    return df_prep

In [ ]:
def train_test_gen_3(df_):
    df_play = df_.copy()

    sample_X = pd.DataFrame([[None,None,None,None,None,None]],columns=['nflId','displayName','jerseyNumber','position','gameId','playId'])

    for i, id in enumerate(df_play['nflId'].unique().astype(int)):
        #sample_X.loc[i,'id'] = str(int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                       str(int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                       str(int(id))
        #sample_X.loc[i,'play'] = str(int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                         str(int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0]))

        sample_X.loc[i,'nflId'] = int(id)
        sample_X.loc[i,'displayName'] = df_play[df_play['nflId']==id]['displayName'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'jerseyNumber'] = int(df_play[df_play['nflId']==id]['jerseyNumber'].fillna(0).drop_duplicates().values[0])
        sample_X.loc[i,'position'] = df_play[df_play['nflId']==id]['position'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'gameId'] = int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])
        sample_X.loc[i,'playId'] = int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0])

        for param in params:
            sample_X.loc[i,param] = df_play[df_play['nflId']==id][param].values[0]

        sample_X.loc[i,'tackle'] = df_play[df_play['nflId']==id]['tackle'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'assist'] = df_play[df_play['nflId']==id]['assist'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'both'] = df_play[df_play['nflId']==id]['tackle'].fillna(0).drop_duplicates().values[0] + \
                                 df_play[df_play['nflId']==id]['assist'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'missed'] = df_play[df_play['nflId']==id]['pff_missedTackle'].fillna(0).drop_duplicates().values[0]

    sample_X = sample_X.fillna(0)

    return sample_X

In [ ]:
def train_test_gen_2(df_):
    fbt = 20  # frames before tackle

    range1 = int(df_['frameId'].max()-fbt)
    range2 = int(df_['frameId'].max()+1)
    frame_list = list(range(range1,range2))

    df_play = df_.copy()

    sample_X = pd.DataFrame([[None,None,None,None,None]],columns=['nflId','displayName','jerseyNumber','gameId','playId'])

    for i, id in enumerate(df_play['nflId'].unique().astype(int)):
        #sample_X.loc[i,'id'] = str(int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                       str(int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                       str(int(id))
        #sample_X.loc[i,'play'] = str(int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])) + '_' + \
        #                         str(int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0]))

        sample_X.loc[i,'nflId'] = int(id)
        sample_X.loc[i,'displayName'] = df_play[df_play['nflId']==id]['displayName'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'jerseyNumber'] = int(df_play[df_play['nflId']==id]['jerseyNumber'].fillna(0).drop_duplicates().values[0])
        sample_X.loc[i,'gameId'] = int(df_play[df_play['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0])
        sample_X.loc[i,'playId'] = int(df_play[df_play['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0])

        for param in params:
            sample_X.loc[i,param] = df_play[df_play['nflId']==id][param].values[0]

        sample_X.loc[i,'tackle'] = df_play[df_play['nflId']==id]['tackle'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'assist'] = df_play[df_play['nflId']==id]['assist'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'both'] = df_play[df_play['nflId']==id]['tackle'].fillna(0).drop_duplicates().values[0] + \
                                 df_play[df_play['nflId']==id]['assist'].fillna(0).drop_duplicates().values[0]

    sample_X = sample_X.fillna(0)

    return sample_X

In [ ]:
def train_test_gen(df_):
    fbt = 20  # frames before tackle

    range1 = int(df_['frameId'].max()-fbt)
    range2 = int(df_['frameId'].max()+1)
    frame_list = list(range(range1,range2))

    df_play_adj = df_[df_['frameId'].isin(frame_list)]

    sample_X = pd.DataFrame(columns=['id'])
    for param in params:
        temp = pd.DataFrame(columns=[param + '_' + str(i) for i in reversed(range(len(frame_list)))])
        sample_X = pd.concat([sample_X,temp],axis=1)

    for i, id in enumerate(df_play_adj['nflId'].unique().astype(int)):
        sample_X.loc[i,'id'] = str(df_play_adj[df_play_adj['nflId']==id]['gameId'].fillna(0).drop_duplicates().values[0]) + '_' + \
                              str(df_play_adj[df_play_adj['nflId']==id]['playId'].fillna(0).drop_duplicates().values[0]) + '_' + \
                              str(id)
        for j, param in enumerate(params):
            for k in range(len(frame_list)):
                sample_X.loc[i,sample_X.columns[j*len(frame_list)+k]] = df_play_adj[df_play_adj['nflId']==id][param].values[k]
        sample_X.loc[i,'tackle'] = df_play_adj[df_play_adj['nflId']==id]['tackle'].fillna(0).drop_duplicates().values[0]
        sample_X.loc[i,'assist'] = df_play_adj[df_play_adj['nflId']==id]['assist'].fillna(0).drop_duplicates().values[0]

    sample_X = sample_X.fillna(0)

    return sample_X

In [ ]:
def calibrate_coords(df_):

    temp = df_.copy()

    # standardize field, refactored from the STRAIN git repo
    temp.loc[temp['playDirection']=='left', 'x'] = 120-temp.loc[temp['playDirection']=='left', 'x']
    temp.loc[temp['playDirection']=='left', 'y'] = (160/3)-temp.loc[temp['playDirection']=='left', 'y']
    temp.loc[temp['playDirection']=='left', 'dir'] = temp.loc[temp['playDirection']=='left', 'dir']+180
    temp.loc[temp['dir']>360, 'dir'] = temp.loc[temp['dir']>360, 'dir']-360
    temp.loc[temp['playDirection']=='left', 'o'] = temp.loc[temp['playDirection']=='left', 'o']+180
    temp.loc[temp['o']>360, 'o'] = temp.loc[temp['o']>360, 'o']-360

    # define start and end events, so we are not calculating strain pre-snap or post-end-of-play
    start_events = ['autoevent_ballsnap', 'ball_snap']
    end_events = ['tackle', 'out_of_bounds']

    # define start and end frames
    temp['is_start'] = 0
    if temp['event'].isin(start_events).sum()==0:
        temp.loc[temp['frameId']==temp['frameId'].min(),'is_start'] = 1
    temp.loc[temp['event'].isin(start_events), 'is_start'] = 1

    temp['is_end'] = 0
    temp.loc[temp['event'].isin(end_events), 'is_end'] = 1
    qwerty = temp[temp['is_end']==1][['playId','frameId','event']].drop_duplicates()

    frame_start, frame_end = temp.loc[temp['is_start']==1, 'frameId'].unique()[0], temp.loc[temp['is_end']==1, 'frameId'].unique()[0]
    # subset data to desired frames
    temp = temp.loc[temp['frameId'].between(frame_start, frame_end)].copy()

    '''
    # calculate velocity towards the ball
    df['v_ball'] = -1.0*(df.loc[:,'football'].sub(df.loc[:,'football'].shift())).div(0.1)

    # create a player ID check, so we don't assign STRAIN values to incorrect players
    df['_shift'] = df.loc[:,'nflId'].shift()
    df.loc[df['nflId']!=df['_shift'], 'v_ball'] = np.nan

    # calculate STRAIN
    #df['strain'] = df['v_ball'].div(df['football'])

    # drop helper columns that are no longer necessary
    df = df.drop(['_shift', 'is_start', 'is_end'], axis=1)
    '''
    temp = temp.drop(['is_start', 'is_end'], axis=1)

    return temp

In [ ]:
def calc_dist_to_football(run_plays_adj, game):
    df = pd.DataFrame()
    for pid in run_plays_adj:

        # subset data down to one play
        play = game.loc[game['playId']==pid].copy()

        for fid in play['frameId'].unique():
            # subset data down to one frame
            frame = play.loc[play['frameId']==fid].copy()

            # make unique positions, as to not duplicate columns based on player position
            frame['pos_unique'] = (frame['position']
                                .add(frame
                                      .groupby('position', as_index=False)
                                      .cumcount()
                                      .add(1)
                                      .dropna()
                                      .astype(str)
                                      .str.replace('.0','', regex=False)
                                      .str.replace('0','', regex=False)))

            # calc distances
            _df = (pd
                .DataFrame(cdist(frame.loc[:, ['x', 'y']],
                                  frame.loc[:, ['x', 'y']]),
                            index=frame['nflId'],
                            columns=frame['pos_unique'].fillna('football')))

            # reset index to pop out nflId into its own column
            _df = _df.reset_index()

            # merge new distance values onto original dataframe
            frame = frame.merge(_df)

            frame['orient_modifier'] = 0.0
            for name in frame.displayName.unique():
                if name != 'football':
                    y_diff = frame.loc[frame['displayName']=='football','y'].values[0] - frame.loc[frame['displayName']==name,'y'].values[0]
                    x_diff = frame.loc[frame['displayName']=='football','x'].values[0] - frame.loc[frame['displayName']==name,'x'].values[0]
                    slope = x_diff/max(0.1,y_diff)
                    orient_to_football = math.degrees(np.arctan(slope))
                    orient_diff = frame.loc[frame['displayName']==name,'o'].values[0] - orient_to_football
                    orient_diff = abs((orient_diff + 180) % 360 - 180)
                    orient_diff_perc = (180-orient_diff)/180
                    frame.loc[frame['displayName']==name,'orient_diff'] = orient_diff_perc

                    dir_diff = frame.loc[frame['displayName']==name,'dir'].values[0] - orient_to_football
                    dir_diff = abs((orient_diff + 180) % 360 - 180)
                    dir_diff_perc = (180-orient_diff)/180
                    frame.loc[frame['displayName']==name,'dir_diff'] = dir_diff_perc
                    #frame.loc[frame['displayName']==name,'orient_modifier'] = 0.5*orient_diff_perc + 0.5

            # concatenate new results into the output dataframe
            df = pd.concat([df,frame])

    return df

In [ ]:
def feature_gen_2(df, plays, pid):

    df_ = pd.merge(df, plays,  how='left', left_on=['gameId','playId'], right_on = ['gameId','playId'])
    nflid = df_[df_['playId']==pid]['ballCarrierId'].unique()[0]
    df_ = df_[df_['playId']==pid]
    df_play = calibrate_coords(df_)
    frame_max = df_play['frameId'].max()

    roll_frames = 1
    if np.any(df_play['event']=='first_contact'):
        #temp['first_contact_to_tackle_diff'] = frame_max - temp[temp['event']=='first_contact']['frameId'].values[0]
        roll_frames = frame_max - df_play[df_play['event']=='first_contact']['frameId'].values[0]

    first_contact_frame = frame_max - roll_frames
    fcf_halfway = round((frame_max - first_contact_frame)/2) + first_contact_frame

    bc = df_play.copy()
    bc = bc[bc['nflId']==nflid]
    x_total = bc['x'].max() - bc['x'].min()
    bc['x_perc'] = (bc['x'] - bc['x'].min())/x_total
    #bc['d_x'] = bc.loc[:,'x'].sub(bc.loc[:,'x'].shift()).div(0.1).fillna(0)
    bc['s_x'] = np.round(np.sin(np.radians(bc['dir'])) * bc['s'],3)
    bc['dS_x'] = bc.loc[:,'s_x'].sub(bc.loc[:,'s_x'].shift(-1)).div(0.1).fillna(0)
    bc['a_x'] = np.round(np.sin(np.radians(bc['dir'])) * bc['a'],3)
    bc['dA_x'] = bc.loc[:,'a_x'].sub(bc.loc[:,'a_x'].shift(-1)).div(0.1).fillna(0)

    for i in range(len(bc)):
        idx = bc.index[i]
        if bc.loc[idx,'y'] > (160/3):
            bc.loc[idx,'y'] = (160/3)
        elif bc.loc[idx,'y'] < 0:
            bc.loc[idx,'y'] = 0

    bc['y_adj'] = np.abs(bc['y'] - (160/6))
    bc['d_y'] = bc.loc[:,'y_adj'].sub(bc.loc[:,'y_adj'].shift(-1)).div(0.1).fillna(0)
    ep1 = 1
    ep2 = 0
    bc['boundary'] = np.exp(bc['y_adj']-ep1*(ep2+160/6)) * bc['d_y']
    bc['dsx'] = bc['dS_x']
    bc = bc[['gameId','playId','frameId','s_x','dS_x','boundary','dA_x','x_perc']]
    df_play = pd.merge(df_play, bc,  how='left', left_on=['gameId','playId','frameId'], right_on = ['gameId','playId','frameId'])
    #set to Defense
    df_play = df_play[df_play['club']==df_play['defensiveTeam']]

    df_play['activation'] = -(1/3)*df_play['football']+1   # linearly increases from zero to 1 once defender within 3 yards of football
    df_play['zero'] = 0
    df_play['point_one'] = 0.1
    df_play['boundary_final'] = df_play[['activation','zero']].values.max(1) * df_play['boundary']
    df_play['football'] = df_play[['football','point_one']].values.max(1)
    df_play['orient_modifier'] = 0.5*df_play['orient_diff'] + 0.5
    df_play['BIG'] = df_play['dS_x'].div(df_play['football'].shift(-1)**2) + df_play['boundary_final']
    df_play['BIG_x'] = df_play['dS_x'].div(df_play['football'].shift(-1)**2)*df_play['x_perc']*df_play['orient_modifier'] + df_play['boundary_final']

    #df_play['BIG_total'] = df_play.groupby('nflId')['BIG'].transform('sum')
    #df_play['BIG_total_sum'] = df_play.groupby('frameId')['BIG_total'].transform('sum')
    #df_play['BIG_total_perc'] = np.round(df_play['BIG_total']/df_play['BIG_total_sum'],5)
    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_total_x'] = df_play.groupby('nflId')['BIG_x'].transform('sum')
    if df_play.groupby('frameId')['BIG_total_x'].transform('min')[0] < 0:
        BIG_total_x_min = abs(df_play.groupby('frameId')['BIG_total_x'].transform('min')[0])
    else:
        BIG_total_x_min = 0
    df_play['BIG_total_x_adj'] = df_play['BIG_total_x'] + BIG_total_x_min
    df_play['BIG_total_sum_x'] = df_play.groupby('frameId')['BIG_total_x_adj'].transform('sum')
    df_play['BIG_total_perc_x'] = df_play['BIG_total_x_adj']/df_play['BIG_total_sum_x']

    '''
    #df_play['BIG_total_perc_x_rank'] = df_play.groupby('frameId')['BIG_total_perc_x'].rank(method="dense", ascending=True)/11
    #df_play['BIG_cumsum'] = df_play.groupby(['nflId','frameId'])['BIG'].cumsum()
    #df_play['BIG_roll'] = df_play['BIG'].rolling(min_periods=1, window=8).sum()
    #zzz['BG2_roll_avg'] = zzz['BIG_2'].rolling(min_periods=1, window=8).mean()
    #df_play['bg_diff'] = df_play.loc[:,'BIG'].sub(df_play.loc[:,'BIG'].shift())
    #df_play['BIG_roll_diff'] = df_play['bg_diff'].rolling(min_periods=1, window=8).sum()

    #df_play['BIG_std'] = df_play.groupby('frameId',group_keys=False)['BIG'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
    #df_play['BIG_std_x'] = df_play['BIG_std']*df_play['x_perc']
    #df_play['BIG_std_total'] = df_play.groupby('nflId')['BIG_std_x'].transform('sum')
    #df_play['BIG_std_total_sum'] = df_play.groupby('frameId')['BIG_std_total'].transform('sum')
    #df_play['BIG_std_total_perc'] = np.round(df_play['BIG_std_total']/df_play['BIG_std_total_sum'],5)

    '''
    df_play['BIG_std_x_roll_20'] = df_play['BIG_std_x'].rolling(min_periods=1, window=20).sum()
    df_play['BIG_std_x_roll_20_last'] = df_play.groupby('nflId')['BIG_std_x_roll_20'].transform('last')
    df_play['BIG_std_x_roll_20_last_sum'] = df_play.groupby('frameId')['BIG_std_x_roll_20_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_std_x_roll_20_last_perc'] = np.round(df_play['BIG_std_x_roll_20_last']/df_play['BIG_std_x_roll_20_last_sum'],5).bfill()
    df_play['BIG_std_x_roll_20_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_std_x_roll_20_last_perc'].rank(method="dense", ascending=True)/11,5)

    df_play['BIG_std_x_roll_10'] = df_play['BIG_std_x'].rolling(min_periods=1, window=10).sum()
    df_play['BIG_std_x_roll_10_last'] = df_play.groupby('nflId')['BIG_std_x_roll_10'].transform('last')
    df_play['BIG_std_x_roll_10_last_sum'] = df_play.groupby('frameId')['BIG_std_x_roll_10_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_std_x_roll_10_last_perc'] = np.round(df_play['BIG_std_x_roll_10_last']/df_play['BIG_std_x_roll_10_last_sum'],5).bfill()
    df_play['BIG_std_x_roll_10_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_std_x_roll_10_last_perc'].rank(method="dense", ascending=True)/11,5)

    df_play['BIG_std_x_roll_5'] = df_play['BIG_std_x'].rolling(min_periods=1, window=5).sum()
    df_play['BIG_std_x_roll_5_last'] = df_play.groupby('nflId')['BIG_std_x_roll_5'].transform('last')
    df_play['BIG_std_x_roll_5_last_sum'] = df_play.groupby('frameId')['BIG_std_x_roll_5_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_std_x_roll_5_last_perc'] = np.round(df_play['BIG_std_x_roll_5_last']/df_play['BIG_std_x_roll_5_last_sum'],5).bfill()
    df_play['BIG_std_x_roll_5_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_std_x_roll_5_last_perc'].rank(method="dense", ascending=True)/11,5)


    #df_play['BIG_roll_std'] = df_play.groupby('frameId',group_keys=False)['BIG_roll'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
    df_play.reset_index(drop=True,inplace=True)
    if df_play.groupby('frameId')['BIG_x'].transform('min')[0] < 0:
        BIG_min = abs(df_play.groupby('frameId')['BIG_x'].transform('min')[0])
    else:
        BIG_min = 0
    df_play['BIG_adj'] = df_play['BIG'] + BIG_min
    df_play['BIG_sum'] = df_play.groupby('frameId')['BIG_adj'].transform('sum')
    df_play['BIG_perc'] = df_play['BIG_adj']/df_play['BIG_sum']
    df_play['BIG_perc_total'] = df_play.groupby('nflId')['BIG_perc'].transform('sum')
    df_play['BIG_perc_total_sum'] = df_play.groupby('frameId')['BIG_perc_total'].transform('sum')
    df_play['BIG_perc_total_perc'] = df_play['BIG_perc_total']/df_play['BIG_perc_total_sum']

    df_play['BIG_perc_roll'] = df_play['BIG_perc'].rolling(min_periods=1, window=roll_frames).sum()
    df_play['BIG_perc_roll_last'] = df_play.groupby('nflId')['BIG_perc_roll'].transform('last')
    df_play['BIG_perc_roll_last_sum'] = df_play.groupby('frameId')['BIG_perc_roll_last'].transform('sum').replace(0,None).bfill()
    df_play['BIG_perc_roll_last_perc'] = (df_play['BIG_perc_roll_last']/df_play['BIG_perc_roll_last_sum']).bfill()

    df_play['BIG_perc_roll_half'] = df_play['BIG_perc'].rolling(min_periods=1, window=max(round(roll_frames/2),1)).sum()
    df_play['BIG_perc_roll_last_half'] = df_play.groupby('nflId')['BIG_perc_roll_half'].transform('last')
    df_play['BIG_perc_roll_last_sum_half'] = df_play.groupby('frameId')['BIG_perc_roll_last_half'].transform('sum').replace(0,None).bfill()
    df_play['BIG_perc_roll_last_perc_half'] = (df_play['BIG_perc_roll_last']/df_play['BIG_perc_roll_last_sum_half']).bfill()

    '''
    df_play['BIG_perc_x_roll_20'] = df_play['BIG_perc_x'].rolling(min_periods=1, window=20).sum()
    df_play['BIG_perc_x_roll_20_last'] = df_play.groupby('nflId')['BIG_perc_x_roll_20'].transform('last')
    df_play['BIG_perc_x_roll_20_last_sum'] = df_play.groupby('frameId')['BIG_perc_x_roll_20_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_perc_x_roll_20_last_perc'] = np.round(df_play['BIG_perc_x_roll_20_last']/df_play['BIG_perc_x_roll_20_last_sum'],5).bfill()
    df_play['BIG_perc_x_roll_20_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_perc_x_roll_20_last_perc'].rank(method="dense", ascending=True)/11,5)

    df_play['BIG_perc_x_roll_10'] = df_play['BIG_perc_x'].rolling(min_periods=1, window=10).sum()
    df_play['BIG_perc_x_roll_10_last'] = df_play.groupby('nflId')['BIG_perc_x_roll_10'].transform('last')
    df_play['BIG_perc_x_roll_10_last_sum'] = df_play.groupby('frameId')['BIG_perc_x_roll_10_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_perc_x_roll_10_last_perc'] = np.round(df_play['BIG_perc_x_roll_10_last']/df_play['BIG_perc_x_roll_10_last_sum'],5).bfill()
    df_play['BIG_perc_x_roll_10_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_perc_x_roll_10_last_perc'].rank(method="dense", ascending=True)/11,5)

    df_play['BIG_perc_x_roll_5'] = df_play['BIG_perc_x'].rolling(min_periods=1, window=5).sum()
    df_play['BIG_perc_x_roll_5_last'] = df_play.groupby('nflId')['BIG_perc_x_roll_5'].transform('last')
    df_play['BIG_perc_x_roll_5_last_sum'] = df_play.groupby('frameId')['BIG_perc_x_roll_5_last'].transform('sum').replace(to_replace=0, method='bfill')
    df_play['BIG_perc_x_roll_5_last_perc'] = np.round(df_play['BIG_perc_x_roll_5_last']/df_play['BIG_perc_x_roll_5_last_sum'],5).bfill()
    df_play['BIG_perc_x_roll_5_last_perc_rank'] = np.round(df_play.groupby('frameId')['BIG_perc_x_roll_5_last_perc'].rank(method="dense", ascending=True)/11,5)
    '''

    df_play['BIG_a'] = df_play['dA_x'].div(df_play['football'].shift(-1)) + df_play['boundary_final']
    df_play['BIG_a_x'] = df_play['dA_x'].div(df_play['football'].shift(-1))*df_play['x_perc']*df_play['orient_modifier'] + df_play['boundary_final']
    df_play['BIG_roll_a'] = df_play['BIG_a'].rolling(min_periods=1, window=8).sum()
    df_play['bg_diff_a'] = df_play.loc[:,'BIG_a'].sub(df_play.loc[:,'BIG_a'].shift())
    df_play['BIG_roll_diff_a'] = df_play['bg_diff_a'].rolling(min_periods=1, window=8).sum()

    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_total_x_a'] = df_play.groupby('nflId')['BIG_a_x'].transform('sum')
    if df_play.groupby('frameId')['BIG_total_x_a'].transform('min')[0] < 0:
        BIG_total_x_min_a = abs(df_play.groupby('frameId')['BIG_total_x_a'].transform('min')[0])
    else:
        BIG_total_x_min_a = 0
    df_play['BIG_total_x_adj_a'] = df_play['BIG_total_x_a'] + BIG_total_x_min_a
    df_play['BIG_total_sum_x_a'] = df_play.groupby('frameId')['BIG_total_x_adj_a'].transform('sum')
    df_play['BIG_total_perc_x_a'] = df_play['BIG_total_x_adj_a']/df_play['BIG_total_sum_x_a']
    df_play['BIG_total_perc_x_rank_a'] = df_play.groupby('frameId')['BIG_total_perc_x_a'].rank(method="dense", ascending=True)/11

    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_min_a'] = df_play.groupby('frameId')['BIG_a_x'].transform('min')
    if df_play.groupby('frameId')['BIG_a_x'].transform('min')[0] < 0:
        BIG_min_a = abs(df_play.groupby('frameId')['BIG_a_x'].transform('min')[0])
    else:
        BIG_min_a = 0
    df_play['BIG_adj_a'] = df_play['BIG_a_x'] + BIG_min_a
    df_play['BIG_sum_a'] = df_play.groupby('frameId')['BIG_adj_a'].transform('sum')
    df_play['BIG_perc_a'] = df_play['BIG_adj_a']/df_play['BIG_sum_a']
    df_play['BIG_perc_total_a'] = df_play.groupby('nflId')['BIG_perc_a'].transform('sum')
    df_play['BIG_perc_total_sum_a'] = df_play.groupby('frameId')['BIG_perc_total_a'].transform('sum')
    df_play['BIG_perc_total_perc_a'] = df_play['BIG_perc_total_a']/df_play['BIG_perc_total_sum_a']

    df_play['BIG_perc_roll_a'] = df_play['BIG_perc_a'].rolling(min_periods=1, window=roll_frames).sum()
    df_play['BIG_perc_roll_last_a'] = df_play.groupby('nflId')['BIG_perc_roll_a'].transform('last')
    df_play['BIG_perc_roll_last_sum_a'] = df_play.groupby('frameId')['BIG_perc_roll_last_a'].transform('sum').replace(0,None).bfill()
    df_play['BIG_perc_roll_last_perc_a'] = (df_play['BIG_perc_roll_last_a']/df_play['BIG_perc_roll_last_sum_a']).bfill()

    df_play['BIG_perc_roll_half_a'] = df_play['BIG_perc_a'].rolling(min_periods=1, window=max(round(roll_frames/2),1)).sum()
    df_play['BIG_perc_roll_last_half_a'] = df_play.groupby('nflId')['BIG_perc_roll_half_a'].transform('last')
    df_play['BIG_perc_roll_last_sum_half_a'] = df_play.groupby('frameId')['BIG_perc_roll_last_half_a'].transform('sum').replace(0,None).bfill()
    df_play['BIG_perc_roll_last_perc_half_a'] = (df_play['BIG_perc_roll_last_a']/df_play['BIG_perc_roll_last_sum_half_a']).bfill()

    #df_play['BIG_x_max_a'] = df_play.groupby('nflId')['BIG_a_x'].transform('max')
    #df_play['BIG_x_max_sum_a'] = df_play.groupby('frameId')['BIG_x_max_a'].transform('sum')
    #df_play['BIG_x_max_perc_a'] = np.round(df_play['BIG_x_max_a']/df_play['BIG_x_max_sum_a'],5)
    #df_play['BIG_x_max_perc_rank_a'] = np.round(df_play.groupby('frameId')['BIG_x_max_perc_a'].rank(method="dense", ascending=True)/11,5)


    df_play['BIG_perc_roll'] = df_play['BIG_perc_roll'].fillna(0)
    df_play['BIG_perc_roll_half'] = df_play['BIG_perc_roll_half'].fillna(0)
    df_play['BIG_x'] = df_play['BIG_x'].fillna(0)
    df_play['BIG_perc_roll_a'] = df_play['BIG_perc_roll_a'].fillna(0)
    df_play['BIG_perc_roll_half_a'] = df_play['BIG_perc_roll_half_a'].fillna(0)
    df_play['BIG_a_x'] = df_play['BIG_a_x'].fillna(0)

    df_play.reset_index(drop=True,inplace=True)
    grouped = df_play[df_play['frameId']==fcf_halfway].groupby('nflId')
    for name, group in grouped:
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_max_halfway'] = group['BIG_perc_roll'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_half_max_halfway'] = group['BIG_perc_roll'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_a_max_halfway'] = group['BIG_perc_roll'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_half_a_max_halfway'] = group['BIG_perc_roll'].max()

    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_perc_roll_max_halfway_sum'] = df_play.groupby('frameId')['BIG_perc_roll_max_halfway'].transform('sum')
    df_play['BIG_perc_roll_max_halfway_perc'] = df_play['BIG_perc_roll_max_halfway']/df_play['BIG_perc_roll_max_halfway_sum']
    df_play['BIG_perc_roll_max_halfway_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_max_halfway_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_half_max_halfway_sum'] = df_play.groupby('frameId')['BIG_perc_roll_half_max_halfway'].transform('sum')
    df_play['BIG_perc_roll_half_max_halfway_perc'] = df_play['BIG_perc_roll_half_max_halfway']/df_play['BIG_perc_roll_half_max_halfway_sum']
    df_play['BIG_perc_roll_half_max_halfway_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_half_max_halfway_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_a_max_halfway_sum'] = df_play.groupby('frameId')['BIG_perc_roll_a_max_halfway'].transform('sum')
    df_play['BIG_perc_roll_a_max_halfway_perc'] = df_play['BIG_perc_roll_a_max_halfway']/df_play['BIG_perc_roll_a_max_halfway_sum']
    df_play['BIG_perc_roll_a_max_halfway_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_a_max_halfway_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_half_a_max_halfway_sum'] = df_play.groupby('frameId')['BIG_perc_roll_half_a_max_halfway'].transform('sum')
    df_play['BIG_perc_roll_half_a_max_halfway_perc'] = df_play['BIG_perc_roll_half_a_max_halfway']/df_play['BIG_perc_roll_half_a_max_halfway_sum']
    df_play['BIG_perc_roll_half_a_max_halfway_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_half_a_max_halfway_perc'].rank(method="dense", ascending=True)/11

    df_play.reset_index(drop=True,inplace=True)
    grouped = df_play[df_play['frameId'].between(first_contact_frame,frame_max)].groupby('nflId')
    for name, group in grouped:
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_max'] = group['BIG_perc_roll'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_half_max'] = group['BIG_perc_roll_half'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_a_max'] = group['BIG_perc_roll_a'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_perc_roll_half_a_max'] = group['BIG_perc_roll_half_a'].max()

    df_play.reset_index(drop=True,inplace=True)
    df_play['BIG_perc_roll_max_sum'] = df_play.groupby('frameId')['BIG_perc_roll_max'].transform('sum')
    df_play['BIG_perc_roll_max_perc'] = df_play['BIG_perc_roll_max']/df_play['BIG_perc_roll_max_sum']
    df_play['BIG_perc_roll_max_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_max_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_half_max_sum'] = df_play.groupby('frameId')['BIG_perc_roll_half_max'].transform('sum')
    df_play['BIG_perc_roll_half_max_perc'] = df_play['BIG_perc_roll_half_max']/df_play['BIG_perc_roll_half_max_sum']
    df_play['BIG_perc_roll_half_max_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_half_max_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_a_max_sum'] = df_play.groupby('frameId')['BIG_perc_roll_a_max'].transform('sum')
    df_play['BIG_perc_roll_a_max_perc'] = df_play['BIG_perc_roll_a_max']/df_play['BIG_perc_roll_a_max_sum']
    df_play['BIG_perc_roll_a_max_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_a_max_perc'].rank(method="dense", ascending=True)/11

    df_play['BIG_perc_roll_half_a_max_sum'] = df_play.groupby('frameId')['BIG_perc_roll_half_a_max'].transform('sum')
    df_play['BIG_perc_roll_half_a_max_perc'] = df_play['BIG_perc_roll_half_a_max']/df_play['BIG_perc_roll_half_a_max_sum']
    df_play['BIG_perc_roll_half_a_max_perc_rank'] = df_play.groupby('frameId')['BIG_perc_roll_half_a_max_perc'].rank(method="dense", ascending=True)/11

    df_play.reset_index(drop=True,inplace=True)
    grouped = df_play[df_play['frameId'].between(first_contact_frame,fcf_halfway)].groupby('nflId')
    for name, group in grouped:
        df_play.loc[df_play['nflId']==int(name),'BIG_x_max_1'] = group['BIG_x'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_x_max_a_1'] = group['BIG_a_x'].max()

    df_play.reset_index(drop=True,inplace=True)
    grouped = df_play[df_play['frameId'].between(fcf_halfway,frame_max)].groupby('nflId')
    for name, group in grouped:
        df_play.loc[df_play['nflId']==int(name),'BIG_x_max_2'] = group['BIG_x'].max()
        df_play.loc[df_play['nflId']==int(name),'BIG_x_max_a_2'] = group['BIG_a_x'].max()

    temp = df_play.copy()
    temp.reset_index(drop=True,inplace=True)
    temp['BIG_x_max_sum_1'] = temp.groupby('frameId')['BIG_x_max_1'].transform('sum')
    temp['BIG_x_max_perc_1'] = temp['BIG_x_max_1']/temp['BIG_x_max_sum_1']
    temp['BIG_x_max_perc_rank_1'] = temp.groupby('frameId')['BIG_x_max_perc_1'].rank(method="dense", ascending=True)/11

    temp['BIG_x_max_sum_a_1'] = temp.groupby('frameId')['BIG_x_max_a_1'].transform('sum')
    temp['BIG_x_max_perc_a_1'] = temp['BIG_x_max_a_1']/temp['BIG_x_max_sum_a_1']
    temp['BIG_x_max_perc_rank_a_1'] = temp.groupby('frameId')['BIG_x_max_perc_a_1'].rank(method="dense", ascending=True)/11

    temp['BIG_x_max_sum_2'] = temp.groupby('frameId')['BIG_x_max_2'].transform('sum')
    temp['BIG_x_max_perc_2'] = np.round(temp['BIG_x_max_2']/temp['BIG_x_max_sum_2'],5)
    temp['BIG_x_max_perc_rank_2'] = np.round(temp.groupby('frameId')['BIG_x_max_perc_2'].rank(method="dense", ascending=True)/11,5)

    temp['BIG_x_max_sum_a_2'] = temp.groupby('frameId')['BIG_x_max_a_2'].transform('sum')
    temp['BIG_x_max_perc_a_2'] = temp['BIG_x_max_a_2']/temp['BIG_x_max_sum_a_2']
    temp['BIG_x_max_perc_rank_a_2'] = temp.groupby('frameId')['BIG_x_max_perc_a_2'].rank(method="dense", ascending=True)/11

    '''
    df_play['BIG_std_a'] = df_play.groupby('frameId',group_keys=False)['BIG_a'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
    df_play['BIG_std_x_a'] = df_play['BIG_std_a']*df_play['x_perc']
    df_play['BIG_std_total_a'] = df_play.groupby('nflId')['BIG_std_x_a'].transform('sum')
    df_play['BIG_std_total_sum_a'] = df_play.groupby('frameId')['BIG_std_total_a'].transform('sum')
    df_play['BIG_std_total_perc_a'] = np.round(df_play['BIG_std_total_a']/df_play['BIG_std_total_sum_a'],5)

    df_play['BIG_std_a'] = df_play.groupby('frameId',group_keys=False)['BIG_a'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
    df_play['BIG_roll_std_a'] = df_play.groupby('frameId',group_keys=False)['BIG_roll_a'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
    df_play['BIG_min_a'] = df_play.groupby('frameId')['BIG_a'].transform('min')
    df_play['BIG_adj_a'] = df_play['BIG_a'] + df_play['BIG_min_a']
    df_play['BIG_sum_a'] = df_play.groupby('frameId')['BIG_adj_a'].transform('sum')
    df_play['BIG_perc_a'] = np.round(df_play['BIG_adj_a']/df_play['BIG_sum_a'],5)
    df_play['BIG_perc_total_a'] = df_play.groupby('nflId')['BIG_perc_a'].transform('sum')
    df_play['BIG_perc_total_sum_a'] = df_play.groupby('frameId')['BIG_perc_total_a'].transform('sum')
    df_play['BIG_perc_total_perc_a'] = np.round(df_play['BIG_perc_total_a']/df_play['BIG_perc_total_sum_a'],5)

    df_play['BIG_perc_x_a'] = df_play['BIG_perc_a'] * df_play['x_perc']
    df_play['BIG_perc_total_x_a'] = df_play.groupby('nflId')['BIG_perc_x_a'].transform('sum')
    df_play['BIG_perc_total_sum_x_a'] = df_play.groupby('frameId')['BIG_perc_total_x_a'].transform('sum')
    df_play['BIG_perc_total_perc_x_a'] = np.round(df_play['BIG_perc_total_x_a']/df_play['BIG_perc_total_sum_x_a'],5)

    df_play['BIG_perc_roll_a'] = df_play['BIG_perc_a'].rolling(min_periods=1, window=20).sum()
    df_play['BIG_perc_rank_a'] = np.round(df_play.groupby('frameId')['BIG_perc_a'].rank(method="dense", ascending=True)/11,5)
    df_play['BIG_perc_rank_roll_a'] = df_play['BIG_perc_rank_a'].rolling(min_periods=1, window=8).mean()
    df_play['BIG_roll_min_a'] = df_play.groupby('frameId')['BIG_roll_a'].transform('min')
    df_play['BIG_roll_adj_a'] = df_play['BIG_roll_a'] + df_play['BIG_roll_min_a']
    df_play['BIG_roll_sum_a'] = df_play.groupby('frameId')['BIG_roll_adj_a'].transform('sum')
    df_play['BIG_roll_perc_a'] = np.round(df_play['BIG_roll_adj_a']/df_play['BIG_roll_sum_a'],5)
    df_play['BIG_roll_perc_rank_a'] = np.round(df_play.groupby('frameId')['BIG_roll_perc_a'].rank(method="dense", ascending=True)/11,5)
    df_play['BIG_roll_perc_rank_roll_a'] = df_play['BIG_roll_perc_rank_a'].rolling(min_periods=1, window=8).mean()
    df_play['BIG_roll_diff_min_a'] = df_play.groupby('frameId')['BIG_roll_diff_a'].transform('min')
    df_play['BIG_roll_diff_adj_a'] = df_play['BIG_roll_diff_a'] + df_play['BIG_roll_diff_min_a']
    df_play['BIG_roll_diff_sum_a'] = df_play.groupby('frameId')['BIG_roll_diff_adj_a'].transform('sum')
    df_play['BIG_roll_diff_perc_a'] = np.round(df_play['BIG_roll_diff_adj_a']/df_play['BIG_roll_diff_sum_a'],5)
    '''

    #temp['bstp_rank'] = np.round(temp.groupby('frameId')['BIG_std_total_perc'].rank(method="dense", ascending=True)/11,5)
    #temp['bptp_rank'] = np.round(temp.groupby('frameId')['BIG_perc_total_perc_x'].rank(method="dense", ascending=True)/11,5)
    #temp['bstp_rank_a'] = np.round(temp.groupby('frameId')['BIG_std_total_perc_a'].rank(method="dense", ascending=True)/11,5)
    #temp['bptp_rank_a'] = np.round(temp.groupby('frameId')['BIG_perc_total_perc_x_a'].rank(method="dense", ascending=True)/11,5)

    #temp['football_min'] = temp.groupby('nflId')['football'].transform('min')
    #temp.loc[temp['football_min']<1,'under_one_yard'] = 1
    #temp.loc[temp['football_min']>=1,'under_one_yard'] = 0

    #temp['first_contact_to_tackle_diff'] = 0
    #if np.any(temp['event']=='first_contact'):
    #    temp['first_contact_to_tackle_diff'] = frame_max - temp[temp['event']=='first_contact']['frameId'].values[0]


    temp.reset_index(drop=True,inplace=True)

    for i in range(len(temp)):
        _nflId = temp.loc[i,'nflId']
        temp.loc[i,'BIG_x_roll_diff'] = temp.loc[i,'BIG_perc_roll_last'] - temp[(temp['frameId']==first_contact_frame)&(temp['nflId']==_nflId)]['BIG_perc_roll'].values[0]

    temp['BIG_x_roll_diff_min'] = temp.groupby('frameId')['BIG_x_roll_diff'].transform('min')
    temp['BIG_x_roll_diff_adj'] = temp['BIG_x_roll_diff'] + temp['BIG_x_roll_diff_min']
    temp['BIG_x_roll_diff_sum'] = temp.groupby('frameId')['BIG_x_roll_diff_adj'].transform('sum')
    temp['BIG_x_roll_diff_perc'] = temp['BIG_x_roll_diff_adj']/temp['BIG_x_roll_diff_sum']

    for i in range(len(temp)):
        _nflId = temp.loc[i,'nflId']
        temp.loc[i,'BIG_x_roll_diff_a'] = temp.loc[i,'BIG_perc_roll_last_a'] - temp[(temp['frameId']==first_contact_frame)&(temp['nflId']==_nflId)]['BIG_perc_roll_a'].values[0]

    temp['BIG_x_roll_diff_min_a'] = temp.groupby('frameId')['BIG_x_roll_diff_a'].transform('min')
    temp['BIG_x_roll_diff_adj_a'] = temp['BIG_x_roll_diff_a'] + temp['BIG_x_roll_diff_min_a']
    temp['BIG_x_roll_diff_sum_a'] = temp.groupby('frameId')['BIG_x_roll_diff_adj_a'].transform('sum')
    temp['BIG_x_roll_diff_perc_a'] = temp['BIG_x_roll_diff_adj_a']/temp['BIG_x_roll_diff_sum_a']


    return temp

In [ ]:
df_play.reset_index(drop=True,inplace=True)
df_play['opt_dist_diff'] = 0
df_play['opt_dir_diff'] = 0
df_play['dist_total'] = 0
for i in range(len(df_play)):
    df_play.loc[i,'dist_total'] += df_play.loc[i,'dis']
    d_vect = {'x': df_play.loc[i,'x'],
              'y': df_play.loc[i,'y'],
              's': df_play.loc[i,'s'],
              'dir': np.radians(df_play.loc[i,'dir'])
              }
    bc_vect = {'x': df_play.loc[i,'x_bc'],
              'y': df_play.loc[i,'y_bc'],
              's': df_play.loc[i,'s_bc'],
              'dir': np.radians(df_play.loc[i,'dir_bc'])
              }
    opt_vect = calc_opt_intersection(d_vect,bc_vect)
    if opt_vect['noop']==True:
        df_play.loc[i,'opt_dist_diff'] = np.nan
        df_play.loc[i,'opt_dir_diff'] = np.nan
    else:
        diff_theta = abs(np.radians(df_play.loc[i,'dir'])-opt_vect['dir'])
        if diff_theta > np.pi:
            diff_theta = 2*np.pi - diff_theta
        df_play.loc[i,'opt_dist_diff'] = 2*df_play.loc[i,'dis']*np.sin(diff_theta/2)
        df_play.loc[i,'opt_dir_diff'] = diff_theta

In [ ]:
def calc_opt_intersection(d_vect,bc_vect):
    opt_vect = find_intercept(d_vect,bc_vect)
    if opt_vect['noop']==True:
        bc_vect_adj = find_boundary_vector_2(bc_vect)
        opt_vect = find_intercept(d_vect,bc_vect_adj)
        return opt_vect
    elif opt_vect['y']<0 or opt_vect['y']>(160/3):
        bc_vect_adj = find_boundary_vector_2(bc_vect)
        opt_vect = find_intercept(d_vect,bc_vect_adj)

    return opt_vect

In [ ]:
def find_intercept(d_vect,bc_vect):
    opt_vect = {'noop': True}
    dist = np.sqrt(((d_vect['x'] - bc_vect['x'])**2) + ((d_vect['y'] - bc_vect['y'])**2))
    dir = np.arctan2((bc_vect['y']-d_vect['y']), (bc_vect['x']-d_vect['x']))
    alpha = np.pi + dir - bc_vect['dir']

    if abs(d_vect['s'] - bc_vect['s']) < 0.01:
        if (np.cos(alpha)) < 0:
            return opt_vect
        else:
            opt_vect['dir'] = (dir + alpha) % (2*np.pi)
            if opt_vect['dir']<0:
                opt_vect['dir'] += 2*np.pi
            bc_vect_m = np.tan(bc_vect['dir'])
            bc_vect_b = bc_vect['y'] - bc_vect_m * bc_vect['x']
            d_vect_m = np.tan(d_vect['dir'])
            d_vect_b = d_vect['y'] - d_vect_m * d_vect['x']
            x_temp = (d_vect_b - bc_vect_b)/(bc_vect_m - d_vect_m)
            opt_vect['x'] = (d_vect_b - bc_vect_b)/(bc_vect_m - d_vect_m)
            y_temp = d_vect_m*opt_vect['x'] + d_vect_b
            opt_vect['y'] = d_vect_m*opt_vect['x'] + d_vect_b
            return opt_vect

    a = d_vect['s']**2 - bc_vect['s']**2
    b = 2 * dist * bc_vect['s'] * np.cos(alpha)
    c = -dist**2
    quadr = b**2 - 4*a*c
    if (quadr < 0):
        return opt_vect
    opt_vect['noop'] = False
    time = (np.sqrt(quadr)-b)/(2*a)
    opt_vect['x'] = bc_vect['x'] + bc_vect['s'] * time * np.cos(bc_vect['dir'])
    opt_vect['y'] = bc_vect['y'] + bc_vect['s'] * time * np.sin(bc_vect['dir'])
    opt_vect['dir'] = np.arctan2(opt_vect['y']-d_vect['y'], opt_vect['x']-d_vect['x'])
    if opt_vect['dir']<0:
        opt_vect['dir'] += 2*np.pi

    return opt_vect

In [ ]:
def find_boundary_vector(bc_vect, opt_vect):
    p1 = (bc_vect['x'],bc_vect['y'])
    p2 = (opt_vect['x'],opt_vect['y'])
    points = [(bc_vect['x'],bc_vect['y']),(opt_vect['x'], opt_vect['y'])]
    x_coords, y_coords = zip(*points)
    A = np.vstack([x_coords,np.ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords,rcond=None)[0]
    if opt_vect['y'] < 0:
        p3 = (-c/m,0)
        dist_to_sideline = math.dist(p1,p3)
        print('dist_to_sideline',dist_to_sideline)
        new_vect_x = (-c/m) - dist_to_sideline
        bc_vect_adj = {'x': new_vect_x,
                       'y': 0,
                       'dir': 0,
                       's': bc_vect['s']}
    else:
        p3 = (((160/3)-c)/m,160/3)
        print('p3',p3)
        dist_to_sideline = math.dist(p1,p3)
        print('dist_to_sideline',dist_to_sideline)
        new_vect_x = (((160/3)-c)/m) - dist_to_sideline
        bc_vect_adj = {'x': new_vect_x,
                       'y': (160/3),
                       'dir': 0,
                       's': bc_vect['s']}

    #find intersection of sideline between bc_vect and x,y
    #find length of path between x,y and intersection of sideline
    #rotate length of path onto sideline, then determine x,y,dir
    #return x,y,dir and recalculate intercept


    return bc_vect_adj

In [ ]:
def find_boundary_vector_2(bc_vect):
    x_bound = (0 - bc_vect['y'])/np.tan(bc_vect['dir']) + bc_vect['x']
    if x_bound >= 0:
        dist_to_sideline = math.dist((bc_vect['x'],bc_vect['y']),(x_bound,0))
        bc_vect_adj = {'x': x_bound - dist_to_sideline,
                       'y': 0,
                       'dir': 0,
                       's': bc_vect['s']}
    else:
        x_bound = (160/3 - bc_vect['y'])/np.tan(bc_vect['dir']) + bc_vect['x']
        dist_to_sideline = math.dist((bc_vect['x'],bc_vect['y']),(x_bound,160/3))
        bc_vect_adj = {'x': x_bound - dist_to_sideline,
                       'y': 160/3,
                       'dir': 0,
                       's': bc_vect['s']}

    return bc_vect_adj

In [ ]:
gid = 2022110610
X_prep = pd.read_csv(f'{project_dir}{gid}prep.csv')
X_prep = X_prep[cols].copy()

In [ ]:
X_prep[['tackle']] = X_prep[['tackle']].fillna(0)

In [ ]:
X_prep.playId.unique()

In [ ]:
pid = 3889
df_pp = X_prep[X_prep['playId']==pid].copy(deep=True)
df_pp.reset_index(drop=True,inplace=True)
df_pp[df_pp['tackle']==1]

In [ ]:
df_pp

In [ ]:
import plotly.graph_objects as go
fig = go.Figure( layout_yaxis_range=[0,53.3], layout_xaxis_range=[0,120])

colors = {"TEN":'#D50A0A',
          "KC":'#003594',
          "football":'#CBB67C'
        }
for team in df_pp.club.unique():
    plot_df = df_pp[(df_pp.club==team)&(df_pp.frameId==1)]
    fig.add_trace(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team))
fig.show()

In [ ]:
colors = {
    'ARI':"#97233F",
    'ATL':"#A71930",
    'BAL':'#241773',
    'BUF':"#00338D",
    'CAR':"#0085CA",
    'CHI':"#C83803",
    'CIN':"#FB4F14",
    'CLE':"#311D00",
    'DAL':'#003594',
    'DEN':"#FB4F14",
    'DET':"#0076B6",
    'GB':"#203731",
    'HOU':"#03202F",
    'IND':"#002C5F",
    'JAX':"#9F792C",
    'KC':"#E31837",
    'LA':"#003594",
    'LAC':"#0080C6",
    'LV':"#000000",
    'MIA':"#008E97",
    'MIN':"#4F2683",
    'NE':"#002244",
    'NO':"#D3BC8D",
    'NYG':"#0B2265",
    'NYJ':"#125740",
    'PHI':"#004C54",
    'PIT':"#FFB612",
    'SEA':"#69BE28",
    'SF':"#AA0000",
    'TB':'#D50A0A',
    'TEN':"#4B92DB",
    'WAS':"#5A1414",
    'football':'#CBB67C'
}

In [ ]:
df_pp[df_pp['tackle']==1]

In [ ]:
df_pp.columns

In [ ]:
df_pp['absoluteYardline'] = 0
yardline = df_pp['yardlineNumber']
df_pp.loc[df_pp['yardlineSide']==df_pp['possessionTeam'].unique()[0],'absoluteYardline'] = yardline
df_pp.loc[df_pp['yardlineSide']==df_pp['defensiveTeam'].unique()[0],'absoluteYardline'] = 50 + (50-yardline)

In [ ]:
animate_play(df_pp).show()

In [ ]:
def animate_play(df_):
#def animate_play(tracking_df, play_df,players,pffScoutingData, gameId,playId):

    df = df_.copy()
    #selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    #tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    #tracking_players_df = pd.merge(tracking_players_df,pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    #selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()
    sorted_frame_list = df.frameId.unique()
    sorted_frame_list.sort()

    # get play General information
    line_of_scrimmage = df.absoluteYardline.values[0]
    first_down_marker = line_of_scrimmage + df.yardsToGo.values[0]
    down = df.down.values[0]
    quarter = df.quarter.values[0]
    gameClock = df.gameClock.values[0]
    playDescription = df.playDirection.values[0]
    gameId = df.gameId.values[0]
    playId = df.playId.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Numbers to Field
        data.append(
            go.Scatter(
                x=np.arange(20,110,10),
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10),
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage],
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker],
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Plot Players
        for team in df.club.unique():
            plot_df = df[(df.club==team)&(df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    if selected_player_df['tackle'].values[0] == 1:
                        hover_text_array.append("nflId:{}<br>displayName:{}<br>Position:{}".format(selected_player_df["nflId"].values[0],
                                                                                          selected_player_df["displayName"].values[0],
                                                                                          selected_player_df["position"].values[0]))
                                                                                          #selected_player_df["pff_role"].values[0]))
                        data.append(go.Scatter(x=selected_player_df["x"], y=selected_player_df["y"],mode = 'markers',marker_color=colors[team],name=team,hovertext=hover_text_array,hoverinfo="text"))
                        data.append(go.Scatter(x=selected_player_df["opt_x"], y=selected_player_df["opt_y"],mode = 'markers',marker_color='blue',name='opt',hoverinfo='none'))
                        data.append(go.Scatter(x=selected_player_df["x_bc"], y=selected_player_df["y_bc"],mode = 'markers',marker_color='black',name='football',hoverinfo='none'))
            #else:
                #data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color='black',name='football',hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )

    return fig

In [ ]:
w = 9
week = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
games_wk9 = week['gameId'].unique().tolist()

In [ ]:
df = df_.copy()

In [ ]:
test = df[(df['gameId'].isin(games_wk9))&(df['displayName']!='football')].copy()
train = df[(~df['gameId'].isin(games_wk9))&(df['displayName']!='football')].copy()

In [ ]:
test[['tackle','assist']] = test[['tackle','assist']].fillna(0)
train[['tackle','assist']] = train[['tackle','assist']].fillna(0)
test['both'] = test['tackle']+test['assist']
train['both'] = train['tackle']+train['assist']

In [ ]:
'football','orient_diff','dir_diff',

In [ ]:
for i in range(len(test.columns)):
    print(test.columns[i])